<a href="https://colab.research.google.com/github/Pooojamangal/datasharing/blob/master/copy_of_Scenario_Playground_(create%2C_upload%2C_Test).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet google-generativeai==0.8.4 anthropic==0.49.0
!pip install --quiet cryptography==43.0.3
import anthropic
import random
import google.generativeai as genai
import datetime
import pandas as pd

import requests
import base64
import json
import time
import threading
import os
import csv
from getpass import getpass
from cryptography.fernet import Fernet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 16.7 MB/s eta 0:00:00


#Login

In [ ]:

base_url = "https://coach-api-gke-dev.coachbots.com"
# base_url = "https://coach-api-gke-prod.coachbots.com"
# base_url = 'https://985a-42-105-234-88.ngrok-free.app' # local


email = input("Enter your email: ")
domain = input("Enter your domain: ")
password = getpass("Enter your password: ")

def webauth_login(domain,email,password,base_url):
    url = f"{base_url}/api/v1/webauth/login/"

    payload = json.dumps({
    "subdomain_prefix": domain,
    "identity_context": {
        "identity_type": "email",
        "value": email
    },
    "password": password
    })
    headers = {
    'Content-Type': 'application/json'
    }
    auth = None
    try:
        response = requests.request("POST", url, headers=headers, data=payload)
        response=response.json()
        print(response)
        auth = f"Bearer {response['access']}"
    except Exception as e:
        print(e)
        auth = None

    return auth


auth = webauth_login(
    domain,email, password, base_url
)
token = auth
if not auth:
  raise Exception("Authentication failed. Cannot proceed with execution.")





In [ ]:
#@title API Keys

# 🔑 Generate and save this key once. Reuse it to decrypt.
def generate_key():
    key = Fernet.generate_key()
    print(f"Secret Key (save this!): {key.decode()}")
    return key

# 🔐 Encrypt text using secret key
def encrypt_text(text: str, key: bytes) -> str:
    f = Fernet(key)
    encrypted = f.encrypt(text.encode())
    return encrypted.decode()
a_en = 'gAAAAABn_gVOnCsyNPsppOx9dheGdc9oB7lwYyaT1tkobYHuUFcCuBxtj8Jsc0lWZnAeJtoSZbN5kfrVjHgNYloixoT-Cq7OV1Uz3ngyHrr27-P8XJtmRS_ad7AwObe6B5sLsZI3884uypgE0gn8Da9AM6u3sOijp6G7_gX-orylNQIfxnV_UT_t-4diM7cyuJMCCXqJH6hgUcLGgnICK0tFnLihNTsVWA=='
b_en = 'gAAAAABn_Phc-3P7-yb8rG1j_cEnzu8YBtgm5iLvFH3voD3pEFD3XebrUDKgpsoK0ZRzlijNElkjsCguRy6JvkK2ksfjD72IFisj6kf3HWhNkU__IUlHgUzKNcUNAZCiVQqGqmcjyg2o'
# 🔓 Decrypt text using secret key
def decrypt_text(token: str, key: bytes) -> str:
    f = Fernet(key)
    decrypted = f.decrypt(token.encode())
    return decrypted.decode()

key = b'Xx84Er3hyl0I53jcS_y4cN8s6ah-ZOaKTyjeYsnwZ1c='

In [ ]:
# @title LLM setup
# if not anthropic_api_key or not google_api_key:
#   raise ValueError("Please enter your API keys in the previous cell")

# Temperature for Anthropic models
anthropic_temperature = 0.3  #@param {type:"slider", min:0, max:1, step:0.1}

# Temperature for Google models
google_temperature = 0 #@param {type:"slider", min:0, max:1, step:0.1}

# Top P for Anthropic models
anthropic_top_p = 0.9 #@param {type:"slider", min:0, max:1, step:0.1}

# Top P for gemini models
google_top_p = 0 #@param {type:"slider", min:0, max:1, step:0.1}

# Maximum output length for Anthropic models
anthropic_max_tokens_to_sample = 2048 #@param {type:"integer"}


# Maximum output length for Google models
google_max_output_tokens = 4000 #@param {type:"integer"}


# Model selection (Anthropic)
anthropic_model = "claude-3-5-haiku-20241022" #@param ["claude-3-haiku-20240307", "claude-3-5-haiku-20241022", "claude-3-opus-20240229", "claude-3-5-sonnet-20241022", "claude-3-5-sonnet-20240620"]

# Model selection (Google)
google_model = "gemini-2.0-flash-lite-001" #@param ["gemini-2.0-flash-001", "gemini-2.0-flash-lite-001", "gemini-1.5-flash-002"]




# Initialize Anthropic client
anthropic_client = anthropic.Client()
anthropic_client.api_key = decrypt_text(a_en,key)


# Initialize Google client
genai.configure(api_key=decrypt_text(b_en,key))


# Example usage (you can adapt this to your needs)
def anthropic_completion(prompt):
    max_retries = 10

    while True:

      try:
          print({"****evaluate_response ":f"trying anthropic for {10 - max_retries + 1} time"})
          response = anthropic_client.messages.create(
                      model=anthropic_model,
                      max_tokens=anthropic_max_tokens_to_sample,
                      temperature=anthropic_temperature,
                      top_p=anthropic_top_p,
                      messages=[
                          {
                              "role": "user",
                              "content": [
                                  {
                                      "type": "text",
                                      "text": prompt
                                  }
                              ]
                          }
                      ],
                  )
          print("anthropic_completion response %s", response)

          return response.content[0].text

      except anthropic.AuthenticationError as e:
          print({"****evaluate_response ":f"failed anthropic for {10 - max_retries + 1} time", "error": e})
          raise e
          break

      except Exception as e:
          print({"****evaluate_response ":f"failed anthropic for {10 - max_retries + 1} time", "error": e})

          if max_retries <= 0:
              print("anthropic_completion error %s", e)
              raise e
              break
          else:
              max_retries -= 1

          time.sleep(random.randint(1,3))



def gemini_completion(prompt,model_name=google_model):

    max_retries = 10

    while max_retries > 0:
        try:
            print(f"Trying API call, attempt {11 - max_retries} for {model_name}")

            model = genai.GenerativeModel(model_name)
            response = model.generate_content(prompt, generation_config={"max_output_tokens": google_max_output_tokens, "temperature": google_temperature, "top_p": google_top_p})

            print(f"Gemini Flash response: {response.text}")
            return response.text

        except genai.types.GoogleAPIError as e:
            print(f"Google API Error on attempt {11 - max_retries}: {e}")
            raise e  # Raise immediately for authentication issues

        except Exception as e:
            print(f"Request failed on attempt {11 - max_retries}: {e}")
            max_retries -= 1
            if max_retries == 0:
                raise e  # Raise the error after max retries
            time.sleep(random.randint(1, 3))  # Randomized delay before retrying

    return response.result

try:
  anthropic_completion("tell me about india in 50 words")
  gemini_completion("tell me about india in 50 words")
except:
  raise ValueError("Please enter correct api keys")

{'****evaluate_response ': 'trying anthropic for 1 time'}
anthropic_completion response %s Message(id='msg_01Ce7Z2wNQGo3kAjHu86Mrpx', content=[TextBlock(citations=None, text="India, a vibrant and diverse nation, is a land of rich cultural heritage, ancient traditions, and rapid economic growth. It is the world's largest democracy, home to diverse religions, languages, and cuisines, and a global hub for technology and innovation. India's unity in diversity is its strength, making it a unique and captivating country.", type='text')], model='claude-3-haiku-20240307', role='assistant', stop_reason='end_turn', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=16, output_tokens=75))
Trying API call, attempt 1 for gemini-2.0-flash-001
Gemini Flash response: India, a vibrant land of ancient history and modern ambition, boasts a diverse tapestry of cultures, languages, and religions. From the Himalayas to the Indian Ocean, it'

# Initializing functions


## Automated Scenario


In [ ]:

import random
import datetime
key = os.getenv("dev_key")
secret = os.getenv("dev_secret")
slack_webhook_url = os.getenv("SLACK_MESSAGE_WEBHOOK_URL")

user_uid = '52927aeb-854c-4573-90c2-bcaf09e6792f'
# user_uid = '37fffb79-75c0-414e-bfa8-0cce75df03f3'

global_file_name = f'Bulk Report Testing - {datetime.datetime.today().strftime("%Y-%m-%d")}.csv'

sample_answers =  [
            "On-time completion of the new office building project. This was a complex project with a tight deadline, but I was able to successfully manage the team and ensure that the project was completed on time and on budget.",
            "Dealing with unexpected weather delays. We experienced a number of unexpected weather delays during the construction of the office building. I was able to manage these delays by adjusting the project schedule and resequencing tasks.",
            "Continue to focus on communication and teamwork. I will continue to focus on communicating effectively with my team members and working collaboratively to achieve our goals."
            ]


errors = []
messages = []

def generate_question_response(title,description,question,is_game=False, previous_conv=None):
    raw_question = question
    if is_game:
      if isinstance(question, str):
        question = json.loads(question)
        options = question['content']['options'].keys()
        resp = random.choice(list(options))
        print(f"Response for question - {raw_question} : ",resp)
        return resp

    else:
      instruction = f'''Title: {title}
                      Description: {description}
                      According to the information provided, please respond to the question below in "a minimum of 30 words and a maximum of 50 words". Maintain a professional tone and respond **directly and insightfully* to the question, avoiding small talk or vague confirmations like “Okay” or “Please ask your questions. etc.”
                  '''
      print('previous conv', previous_conv)
      if previous_conv != None:
        question = f"Previous Conversation: {previous_conv}\n\n{question}"
      else:
        question = f"Question: {question}"

      for i in range(3):
        try:
          resp = requests.post(f'{base_url}/api/v1/documents/get-prompt-response/',data=json.dumps({
                      'prompt': question,
                      'instruction': instruction,
                  }), headers={
            'Content-Type': 'application/json'
          })
          # resp = requests.get(f'{base_url}/api/v1/documents/get-prompt-response/?prompt={question}&instruction={instruction}')
          print(resp.text)
          resp.raise_for_status()
          resp =  resp.json()
          print(f"Response for question - {raw_question} : ",resp['response_text'])
          return resp['response_text']
        except Exception as e:
          print(e)
          if i+1 == 3:
            raise e
          time.sleep(3)


def upload_file():
  url = f"{base_url}/api/v1/documents/upload/"
  file_path = "response.mp3"
  user_id = user_uid

  # Construct the form data
  form_data = {
      "owner_type": "user",
      "owner_id": user_id,
      "display_name": "automated samplel response",
      "doc_type": "AUDIO_ANSWER",
      "actions_pipeline[0]action": "transcribe",
      "actions_pipeline[0]context": "null"
  }

  # Open the file and add it to the form data
  with open(file_path, 'rb') as file:
      files = {'file': (file.name, file, 'multipart/form-data')}
      response = requests.post(url,headers=get_quick_headers(), data=form_data, files=files)

  # Handle response
  if response.status_code == 201:
      print("File uploaded successfully")
      response = response.json()
      print(response)
      return response
  else:
      print("Error:", response.text)






def get_doc_url(doc_id):
  url = f"{base_url}/api/v1/documents/{doc_id}/url"
  response = requests.post(url,headers=get_quick_headers())
  response = response.json()
  # print(response)
  return response


def text_to_speech(text):
  doc = requests.get(f"{base_url}/api/v1/test-responses/get-text-to-speech/?text={text}")
  with open('response.mp3', 'wb') as f:
    f.write(doc.content)

def get_text_audio_url(text):
  text_to_speech(text)


  doc = upload_file()
  doc_id = doc['uid']

  doc_url = get_doc_url(doc_id)['url']
  print('Doc URL :',doc_url)
  return doc_url


def send_slack_message(data):
    url = slack_webhook_url

    # data.update({"message":"Testing slack webhook"})

    payload = json.dumps({
        "text": json.dumps(data, default=str)
    })

    headers = {
        'Content-type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    try:
        response.raise_for_status()
    except Exception as e:
        print("failed to send to slack data: %s, err: %s", data, str(e))


def create_basic_auth_token(key: str, secret: str) -> str:
    return base64.b64encode(f"{key}:{secret}".encode("utf-8")).decode()


def get_headers(basic_auth_token):
        return {
            "Authorization": basic_auth_token
        }

def get_quick_headers():
    return get_headers(token)

def create_or_get_account():
    try:
        response = requests.post(base_url + '/api/v1/accounts/', json={
                "user_context": {
                    "name": "test-user",
                    "role": "member",
                    "user_attributes": {
                        "tag": "deepchat_profile",
                        "attributes": {
                                "username": "test-user611",
                            }
                    }
                },
                "identity_context": {
                    "identity_type": "deepchat_unique_id",
                    "value": f"test-user611"
                }
            },
            headers=get_quick_headers()
        )

        response =  response.json()
        user_uid = response['uid']
        return response
    except Exception as e:
        errors.append("Error in creating account: " + str(e.args))


def get_test_from_code(code='Q7JS0OL'):
    try:
        response = requests.get(base_url + f'/api/v1/tests/?test_code={code}', headers=get_quick_headers())
        if response.status_code == 200:
            messages.append("Got test from code successfully")
        else:
            errors.append("Error in getting test from code: ")
        print(response.json())
        return response.json()
    except Exception as e:
        errors.append("Error in getting test from code: " + str(e.args))
        return None

def cancel_previous_session():
    try:
        response = requests.get(base_url + "/api/v1/test-attempt-sessions/cancel-test-sessions/", json={
                "user_id": user_uid
                },  headers=get_quick_headers()
        )

        if response.status_code == 200:
            messages.append("Cancelled previous session successfully")
        else:
            errors.append("Error in cancelling previous session: ")

        response =  response.json()
        print(response)
        return response
    except Exception as e:
        errors.append("Error in cancelling previous session: " + str(e.args))

def create_test_session(test_id, participant_id):
    try:
        response = requests.post(base_url + f'/api/v1/test-attempt-sessions/', json={
                "test_id": test_id,
                "participant_id": participant_id
            },
            headers=get_quick_headers()
        )

        if response.status_code == 201:
            messages.append("Created test session successfully")
        else:
            errors.append("Error in creating test session: ")

        # print("create test session status code: ", response.status_code)

        response =  response.json()
        return response
    except Exception as e:
        print(e)
        errors.append("Error in creating test session: " + str(e.args))


def submit_response(test_attempt_session_id, question_id, response_text, response_url):
    try:
        response = requests.post(base_url + f'/api/v1/test-responses/', json={
                "test_attempt_session_id": test_attempt_session_id,
                "question_id": question_id,
                "response_text": response_text,
                "response_file": response_url
            },
            headers=get_quick_headers()
        )

        if response.status_code == 201:
            messages.append("Submitted response successfully")
        else:
            errors.append("Error in submitting response: ")
        # print("submit response status code: ", response.status_code)

        response =  response.json()
        print(response)
        return response
    except Exception as e:
        print(e)
        errors.append("Error in submitting response: " + str(e.args))


def get_report_url(session_id, interaction_id,report_type='interactionSessionReport'):
    try:
        response = requests.post(base_url + f'/api/v1/frontend-auth/get-report-url/', json={
                "user_id": user_uid,
                "report_type": report_type,
                "session_id": session_id,
                "interaction_id": interaction_id,
                "test_attempt_session_id": session_id,
            },
            headers=get_quick_headers()
        )

        if response.status_code == 200:
            messages.append("Got Report url successfully")
        else:
            errors.append("Error in getting report url: ")

        response =  response.json()
        # print("GET REPORT URL RESPONSE : ",response)

        return response
    except Exception as e:
        errors.append("Error in getting report url: " + str(e.args))


def send_report_email(test_attempt_session_id, report_url):
    try:
        response = requests.post(base_url + f'/api/v1/test-attempt-sessions/send-report-email/', params={
                "test_attempt_session_id": test_attempt_session_id,
                "report_url": report_url,
                "is_whatsapp": False
            },
            headers=get_quick_headers()
        )

        if response.status_code == 200:
            messages.append("Sent report email successfully")
        else:
            errors.append("Error in sending report email: ")

        response =  response.json()
        print(response)
        return response
    except Exception as e:
        errors.append("Error in sending report email: " + str(e.args))

# print(get_headers(create_basic_auth_token(key, secret)))

def attempt_test(test, is_audio=False):
    create_or_get_account()

    # test = get_test_from_code(test_code)

    test_id = test['results'][0]['uid']
    questions = test['results'][0]['questions']
    no_of_questions = len(questions)

    cancel_previous_session()
    session = create_test_session(test_id, user_uid)
    print(session)
    session_id = session['uid']

    print('Session created with id: ', session_id)
    # session_id = '6679a137-58fd-4ca0-ab42-ffb2c7e6e1f5'

    start = time.time()
    for i in range(no_of_questions):
        # answer = sample_answers[i]
        answer = generate_question_response(test['results'][0]['title'],test['results'][0]['description'],questions[i]['question'])
        # if i + 1 == no_of_questions:
        file_url = ""
        if is_audio:
            file_url = get_text_audio_url(answer)
            answer = ""

        print(f"####### answer: {answer},,,, file_url: {file_url}")
        submit_response(session_id, questions[i]['uid'], answer, file_url)
        """ else:
            threading.Thread(target=submit_response, args=(session_id, questions[i]['uid'], answer, "")).start() """
    end = time.time()
    print(f"Time taken to submit all responses", end - start, " seconds")
    messages.append("Time taken to submit all responses " + ": " + str(end - start) + " seconds")

    report_type = "interactionSessionReport"
    if test['results'][0]['scenario_case'] == "psychometric":
      report_type = "personalityPsychomatricReport"
    report = get_report_url(session_id, test_id, report_type=report_type)
    print(f"Report URL: {report['url']}")
    messages.append("Report url: " + report['url'])

    # time.sleep(20)
    send_report_email(session_id, report['url'])
    return report['url'], session_id



def attempt_dynamic_test(test, is_audio=False):
    create_or_get_account()

    if test['results'][0]['test_type'] not in ["orchestrated_conversation", "dynamic_discussion", "dynamic_discussion_thread"]:
      print("!!U!Un!Uns!Unsu!Unsup!Unsupp!Unsuppo!Unsuppor!Unsupport!Unsupporte!Unsupported!Unsupported Test type ")

    test_id = test['results'][0]['uid']
    questions = test['results'][0]['questions']
    no_of_questions = len(questions)

    print("No of questions: ", no_of_questions)
    print("Questions: ", questions)
    print("scenario_case", test['results'][0]['scenario_case'])


    cancel_previous_session()
    session = create_test_session(test_id, user_uid)
    session_id = session['uid']
    print('Session created with id: ', session_id, session)
    # session_id = '6679a137-58fd-4ca0-ab42-ffb2c7e6e1f5'

    question_text = ""

    start = time.time()
    previous_conv = ""

    if test['results'][0]['scenario_case'] == 'game':
      question_text = session['next_question_text']
      print(f'question_text game :  {question_text}')
      while True:
        answer = generate_question_response(test['results'][0]['title'],test['results'][0]['description'],question_text,is_game=True)
        resp = submit_response(session_id, "sdkfj", answer, "")
        print(f'resp:{resp}')
        question_text = resp['question_text']
        if 'end_message' in question_text:
          question_text = json.dumps(resp['question_text'])

          break
    else:
      question_text = questions[0]['question']

      orchestrated_conversation_details = test['results'][0]['orchestrated_conversation_details']
      print('orc conv', orchestrated_conversation_details)
      for i in range(no_of_questions):
          if i == 0:
            if orchestrated_conversation_details.get('start_with_user', None) == None:
                question_text = "\n".join(orchestrated_conversation_details['initial_messages'])
                previous_conv += f"{question_text}\n"

          if questions[i]['question_for'] == "user":

              answer = generate_question_response(test['results'][0]['title'],test['results'][0]['description'],question_text,previous_conv=previous_conv)
              file_url = ""
              previous_conv += f"User: {answer}\n"
              if is_audio:
                  file_url = get_text_audio_url(answer)
                  answer = ""
              submit_response(session_id, questions[i]['uid'], answer, file_url)
          else:
              resp = submit_response(session_id, questions[i]['uid'], "", "")
              question_text = resp['response_text']
              previous_conv += f"{questions[i]['question_for']}: {question_text}\n"

    end = time.time()
    print(f"Time taken to submit all responses", end - start, " seconds")
    messages.append("Time taken to submit all responses " + ": " + str(end - start) + " seconds")

    if test['results'][0]['scenario_case'] != 'game':
      test_type = test['results'][0]['test_type']
      report_type =  "dynamicDiscussionReport"

      if test_type == "orchestrated_conversation":
        report_type = "meetingAnalysisReport"
      report = get_report_url(session_id, test_id,report_type)

      print(f"Report URL: {report['url']}")
      messages.append("Report url: " + report['url'])

      # time.sleep(20)
      send_report_email(session_id, report['url'])
      return report['url'], session_id

    else:
      return question_text, session_id


def to_batches(data, batch_size):
    batches = []
    for i in range(0, len(data), batch_size):
        batches.append(data[i:i + batch_size])
    return batches

# Function to check if the CSV file exists and write header if not
def initialize_csv(file_name, fields= ["Test Code","TEST TITLE","Client Name", "Test Type","Report Link", "Success" ,"Error", "Feedback script","Video Script"]):
    from google.colab import drive
    import os

    drive.mount('/content/drive')

    # Create a folder in Drive
    folder_path = '/content/drive/MyDrive/Collab Downloads'
    os.makedirs(folder_path, exist_ok=True)

    file_path = os.path.join(folder_path, file_name)

    # Check if file already exists
    file_exists = os.path.isfile(file_path)

    # Open the file in append mode and write header if not exists
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(fields)



# Function to append a row to the CSV file
def append_to_csv(file_name, row):
    folder_path = '/content/drive/MyDrive/Collab Downloads'

    file_path = os.path.join(folder_path, file_name)
    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(row)

def get_test_by_filter(filter_params:dict):


    url = f"{base_url}/api/v1/tests/get-tests-by-filter/"
    headers=get_quick_headers()
    print(base_url)
    try:
      response = requests.request("GET",url , headers=headers, params=filter_params)
      print(response)
      print(f"get test by filter {response.json()}")
      return response.json()
    except Exception as e:
      print(f"failed to get test by filter: {e}")



def simulate_scenarios(test_codes,report_csv):
  for code in test_codes:
      test = get_test_from_code(code)
      print("GET TEST FROM CODE : ",test)

      test_data = []
      session_id = ''

      if test is None:
        print(f"!!!!!!!! EROOR: Invalid Test CODE : {code}")
        append_to_csv(report_csv,[code,"","","", "",False, "Invalid test code","", ""])
        continue
      if len(test['results']) == 0:
        print(f"!!!!!!!! EROOR: Invalid Test CODE : {code}")
        append_to_csv(report_csv,[code,"","","","", False, "Invalid test code","", ""])
        continue
      print("Test details : ",test)

      if test['results'][0]['test_type'] in UNSUPPORTED_TEST_TYPES:
        print("!!!!! ERROR : Unsupported Test type.")
        append_to_csv(report_csv, [code,"","","","", False, "Unsupported test type","",""])
        continue

      test_data.append(code)
      test_data.append(test['results'][0]['title'])
      test_data.append(test['results'][0]['client_name'])
      test_data.append(test['results'][0]['test_type'])


      if test['results'][0]['test_type'] in ["orchestrated_conversation", "dynamic_discussion", "dynamic_discussion_thread"]:
          print(f'*************** Attempting TEST: {code} in TEXT MODE ***************')
          try:
            url, session_id = attempt_dynamic_test(test,IS_AUDIO)
            test_data.append(url)
          except Exception as e:
            print(f"!!!!!!!!!!!!!! Failed To attempt TEST: {code}, REASON: {e}")
          print(f"**********************************************************************************************",end="\n\n\n\n")

          # if test['results'][0]['interaction_mode'] == 'any':
          #   print(f'*************** Attempting TEST: {code} in AUDIO MODE ***************')
          #   try:
          #     url = attempt_dynamic_test(test,True)
          #     test_data.append(url)
          #   except Exception as e:
          #     print(f"!!!!!!!!!!!!!! Failed To attempt TEST: {code}, REASON: {e}")
          #   print(f"**********************************************************************************************",end="\n\n\n\n")
      else:
          print(f'****************** Attempting TEST: {code} in TEXT MODE ************************** ')
          try:
            url, session_id = attempt_test(test,IS_AUDIO)
            test_data.append(url)
          except Exception as e:
            print(f"!!!!!!!!!!!!!! Failed To attempt TEST: {code}, REASON: {e}")
          print(f"**********************************************************************************************",end='\n\n\n\n')

          # if test['results'][0]['interaction_mode'] == 'any':
          #   print(f'****************** Attempting TEST: {code} in AUDIO MODE ************************** ')
          #   try:
          #     url = attempt_test(test,True)
          #     test_data.append(url)
          #   except Exception as e:
          #     print(f"!!!!!!!!!!!!!! Failed To attempt TEST: {code}, REASON: {e}")
          #   print(f"**********************************************************************************************",end='\n\n\n\n')

      test_data.append(True)
      test_data.append(None)
      if test['results'][0]['test_type'] in ["orchestrated_conversation", "dynamic_discussion", "dynamic_discussion_thread"]:
        report = get_report(session_id, 'dynamic',report_only=True)
        test_data.append(report.get('feedback_video_script'))
        test_data.append(report.get('video_script'))
      else:
        report = get_report(session_id, 'test',report_only=True)
        test_data.append(report.get('feedback_video_script'))
        test_data.append(report.get('video_script'))
      print(test_data)
      append_to_csv(report_csv,test_data)


def generate_test(TEST_CODES, IS_AUDIO, file_name=global_file_name):
  initialize_csv(file_name)
  UNSUPPORTED_TEST_TYPES= ["mcq","dynamic_mcq","coaching"]
  session_ids = []

  for code in TEST_CODES:
    test_data = []
    test = get_test_from_code(code)
    print(test)
    if test is None:
        print(f"!!!!!!!! EROOR: Invalid Test CODE : {code}")
        append_to_csv(file_name,[code,"","","", "", False, "Invalid test code", "",""])
        continue
    if len(test['results']) == 0:
      print(f"!!!!!!!! EROOR: Invalid Test CODE : {code}")
      append_to_csv(file_name,[code,"","","","", False, "Invalid test code", "", ""])
      continue
    print("Test details : ",test)

    if test['results'][0]['test_type'] in UNSUPPORTED_TEST_TYPES:
      print("!!!!! ERROR : Unsupported Test type.")
      append_to_csv(file_name, [code,"","","", "",False, "Unsupported test type","", ""])
      continue


    test_data.append(code)
    test_data.append(test['results'][0]['title'])
    test_data.append(test['results'][0]['client_name'])
    test_data.append(test['results'][0]['test_type'])

    session_id = ''
    if test['results'][0]['test_type'] in ["orchestrated_conversation", "dynamic_discussion", "dynamic_discussion_thread"]:
        print(f'*************** Attempting TEST: {code} in TEXT MODE ***************')
        try:
          url,session_id = attempt_dynamic_test(test,False)
          session_ids.append(session_id)
          test_data.append(url)
        except Exception as e:
          print(f"!!!!!!!!!!!!!! Failed To attempt TEST: {code}, REASON: {e}")
          raise e
        print(f"**********************************************************************************************",end="\n\n\n\n")

        # if (test['results'][0]['interaction_mode'] == 'any' and IS_AUDIO) and test['results'][0]['scenario_case'] != 'game':
        #   print(f'*************** Attempting TEST: {code} in AUDIO MODE ***************')
        #   try:
        #     url,session_id = attempt_dynamic_test(test,True)
        #     session_ids.append(session_id)

        #     test_data.append(url)
        #   except Exception as e:
        #     print(f"!!!!!!!!!!!!!! Failed To attempt TEST: {code}, REASON: {e}")
        #   print(f"**********************************************************************************************",end="\n\n\n\n")
        # else:
        #   test_data.append("")
    else:
        print(f'****************** Attempting TEST: {code} in TEXT MODE ************************** ')
        try:
          url, session_id = attempt_test(test,False)
          session_ids.append(session_id)
          test_data.append(url)
        except Exception as e:
          print(f"!!!!!!!!!!!!!! Failed To attempt TEST: {code}, REASON: {e}")
          raise e
        print(f"**********************************************************************************************",end='\n\n\n\n')
        # if test['results'][0]['interaction_mode'] == 'any' and IS_AUDIO:
        #   print(f'****************** Attempting TEST: {code} in AUDIO MODE ************************** ')
        #   try:
        #     url,session_id = attempt_test(test,True)
        #     session_ids.append(session_id)

        #     test_data.append(url)
        #   except Exception as e:
        #     print(f"!!!!!!!!!!!!!! Failed To attempt TEST: {code}, REASON: {e}")
        #   print(f"**********************************************************************************************",end='\n\n\n\n')
        # else:
        #   test_data.append("")

    test_data.append(True)
    test_data.append(None)
    if test['results'][0]['test_type'] in ["orchestrated_conversation", "dynamic_discussion", "dynamic_discussion_thread"]:
      report = get_report(session_id, 'dynamic',report_only=True)
      test_data.append(report.get('feedback_video_script'))
      test_data.append(report.get('video_script'))
    else:
      report = get_report(session_id, 'test',report_only=True)
      test_data.append(report.get('feedback_video_script'))
      test_data.append(report.get('video_script'))



    print(test_data)
    append_to_csv(file_name,test_data)

  print("CSV file written successfully!")
  print(errors)

  return session_ids

def download_file(file_name, is_drive=True):
  from google.colab import files
  import shutil
  if is_drive:
    # Path to file in Google Drive
    gdrive_path = f'/content/drive/MyDrive/Collab Downloads/{file_name}'

    # Copy to /content/
    shutil.copy(gdrive_path, f'/content/{file_name}')

    # Download to your local machine
    files.download(f'/content/{file_name}')
  else:
    files.download(file_name)



def create_test(information, creator_user_id=None, use_anthropic=False, flavour='normal',is_micro=True, previous_session_id=None, custom_prompt=None):
  for _ in range(3):
    try:
      url = f"{base_url}/api/v1/tests/get_or_create_test_scenarios_by_site/"
      data = {
          "url": None,
          "mode": "A",
          "access_token": auth,
          "creator_user_id": creator_user_id,
          "flavour": flavour,
          "is_micro": is_micro,
          "information":json.dumps({'title': "",'data':{'information':information}}),
          "use_anthropic": use_anthropic,
          "previous_session_id": previous_session_id,
          "custom_prompt": custom_prompt

      }

      headers = {
      'Content-Type': 'application/json',
      **get_quick_headers()
    }

      response = requests.post(url, data=json.dumps(data), headers=headers)
      print(response.json())

      print(response.json()[0]['test_code'])
      return response.json()
    except Exception as e:
      print(e)
      raise e
      continue


def get_report(session_id, test_type, report_only=False):
  url = f"{base_url}/api/v1/test-attempt-sessions/{session_id}/"
  if test_type == 'test':
    url+= 'report-data/'
  elif test_type == 'dynamic':
    url += 'meeting-report-data/'

  headers = {
  'Content-Type': 'application/json',
  'Authorization': f'Basic {auth}'
}

  response = requests.get(url,headers=get_quick_headers())
  print(response.json())
  if response.ok:
    data = response.json()
    if report_only:
      return data['data']
    if test_type == 'test':
      r = {
      'Title': data['data']['title'],
      'description': data['data']['test_description'],
      'question And Answer': data['data']['qa'],
      'skills_graph_data': data['data']['skills_graph_data'],
      'skills_explanation': data['data']['skills_explanation'],
      'feedback_summary': data['data']['feedback_summary'],
      'skill_summary': data['data']['skill_summary'],
      'culture_graph_data': data['data']['culture_graph_data'],
      'culture_skills_explanation': data['data']['culture_skills_explanation'],
      'competency_data': data['data']['competency_data']
    }

    elif test_type == 'dynamic':
      r ={
      'Title': data['data']['title'],
      'description': data['data']['test_description'],
      'objective': data['data']['objective'],

      'question And Answer': data['data']['chat_conversation'],
      'skills_graph_data': data['data']['skills_rating'],
      'skills_explanation': data['data']['skills_explanation'],
      'feedback_summary': data['data']['feedback_summary'],
      'skill_summary': data['data']['skill_summary'],
      'culture_graph_data': data['data']['culture_skills'],
      'culture_skills_explanation': data['data']['culture_skills_explanation'],
      'competency_data': data['data']['competency_data']
    }

    report_data = ''
    for key, value in r.items():
      report_data += f'{key}: {value}\n'
    return report_data

  else:
    raise Exception('Failed to get report data')


import requests

def upload_scenario(myfile, email, password, domain_prefix, test_type='static'):
    url = f"{base_url}/api/upload-scenarios/"

    # Form data
    data = {
        "email": email,
        "password": password,
        "client_domain_prefix": domain_prefix,
        "test_type": test_type
    }

    # File upload
    files = {
        "myfile": myfile  # should be a file-like object or tuple (filename, fileobj, mimetype)
    }

    try:
        response = requests.post(url, data=data, files=files)
        print(response.text)
        response.raise_for_status()  # Raise error for bad responses
        print("✅ Upload successful!")
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Upload failed: {e}")
        return {"error": str(e)}



def create_and_attempt_test(csv_file,email,password,domain,test_type,file_name=None):
  uploaded_test = upload_scenario(csv_file,email,password,domain,test_type)
  if not uploaded_test['success']:
    raise Exception(f"Failed to generate all test: {uploaded_test['errors']}")

  test_codes= []
  for value in uploaded_test['file_content'].split('\n'):
    title = value.split(':')[0]
    test_code = value.split(',')[-1].strip()
    print(title, test_code)
    if test_code.startswith('Q'):
      test_codes.append(test_code)

  print(test_codes)

  generate_test(test_codes, False)




# base_url = 'https://coach-api-gke-dev.coachbots.com'
# base_url = 'https://1ee0-106-76-71-70.ngrok-free.app'


## Scenario Creator Functions

### **Prompts**

In [ ]:
def video_script_prompt(objective,type_of_prompt='normal_video_script'):
  today_date = datetime.datetime.now().strftime("%Y-%m-%d")
  return f"""
    You are tasked with creating a video script. Please adhere *strictly* to the following requirements:

    **INPUT (User Must Provide):**
    1.  `\"{objective}\"`: Clearly define the central theme, argument, or scenario the video script should focus on. This is the core message Jay will deliver.
    2.  `[{today_date}]`: Specify the current month and year (e.g., "October 2024"). This will be used to define the end date for case study searches.

    **OUTPUT REQUIREMENTS:**

    1.  **Format:** A single-speaker narration script for a video.
    2.  **Speaker:** The script must be written from the perspective of "Jay," an AI Coach.
    3.  **Opening Line:** The script *must* begin *exactly* with: `Hey this is Jay. Your AI Coach.` (No variations allowed).
    4.  **Tone & Style:**
        *   **Opinionated & Strong:** Present a clear, potentially controversial stance on the `\"{objective}\"`.
        *   **Engaging Hook:** Start strong to grab attention immediately. Aim for language that sparks curiosity or challenges common assumptions, suitable for platforms like LinkedIn or YouTube, but maintaining professionalism for an enterprise audience.
        *   **Authoritative & Coaching:** Jay should sound knowledgeable and directive, as expected from a coach in a management development context.
    5.  **Content - Case Studies:**
        *   Include **three (3) real-life examples/case studies** that strongly illustrate the point being made about the `\"{objective}\"`.
        *   **Recency:** These examples *must* reference events, decisions, or published outcomes that occurred between **January 1, 2023, and the `[April,30 2025]` provided by the user.**
        *   **Verifiability:** For each example, include specific details (e.g., company name, specific initiative, reported outcome, relevant timeframe) and **cite a verifiable public reference** (e.g., reputable news article link, official company press release link, widely reported event). The reference should allow someone to cross-verify the claim. *Note: Finding 3 perfectly fitting, publicly referenced examples within this specific recent timeframe can be challenging; prioritize accuracy and relevance.*
    6.  **Target Audience:** The script's complexity, language, and examples should be suitable for **Management Development Programs within enterprise companies.**
    7.  **Length & Pacing:** Aim for a script that can be delivered naturally in **under 2 minutes**. This typically translates to approximately **250-300 words**. Keep all sentences brief—ideally under **10-15 words.** *Do NOT mention the word count anywhere in the generated script itself.*
    8.  **Exclusions:**
        *   Do **NOT** include any title for the video script.
        *   Do **NOT** include typical YouTube/video calls to action (e.g., "like, share, subscribe," "hit the bell icon," "link in description").
        *   Do **NOT** include any closing remarks or sign-offs (e.g., "Thanks for watching," "See you next time," "This has been Jay"). The script should end after the main content/final point is delivered.

      """
def get_scenario_prompt(scenario_type,information,skill_count=2,question_count=3,create_skill=False):
    prompt = ""
    if scenario_type == 'normal_static':
      prompt = """
        \n\nHuman:
        {Information} -
        %s -
        Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
        Description - Define the situation and the problem. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails. description in markdown la.
        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
        Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
        Title must follow this Format:
            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
            Example format:
                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"

        Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the situation. NEVER provide a response to the questions.
        Custom prompt - With each question, add a prompt that would ask feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions.

        KLP - With each question add one or two line takeaway for providing feedback. The takeaways should be related to the question it is provided with.
        KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. Split compound skill words into separate words with proper capitalization. Each question must have a completely unique set of skills — no skill should repeat in any other question.
        Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        At the end of the scenario description please provide a short executive summary that contains the data driven background information NOT captured the situation that the user can levearge to answer any questions related to the scenario.
        In every response, you must:
        Clearly state your role as X.
        Identify Y as the person asking
        The Question, Custom Prompt, KLP, KLS should be numbered.
        Here the format looks like :
        "Title:",
        "Description:”,
        “Statement:",
        "Background:",
        "Question 1:",
        "Prompt 1:",
        "Takeaway 1:" ,
        "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
        'The Question, Prompt, Takeaway, Skills should be numbered.'
        NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
        NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
        NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated. Skill must be only maixmum two words.
        NOTE: KLS - The Skills should be highly relevant to the scenario and {skill_domain}(90 percent or more relevant).
        NOTE: "Rating" must be included.
        NOTE : Make sure the simulation is very advanced and tough.
        NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        NOTE: Never miss Title, Description, Statement and other variables.
        NOTE: Do not mention "X" or "Y".
        NOTE: must Follow the OUTPUT Format.
        \n\nAssistant:

    """
    elif scenario_type == 'whatsapp_normal_static':
      prompt = '''
      \n\nHuman:
        {Information} - %s-
        Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
        Description - Define the situation and the problem within {50} words. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration. It shall always remain between two specific individuals, as determined by the context provided in the {information}.
        Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
        Title must follow this Format:
            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
            Example format:
                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
        Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the problem. NEVER respond to the questions.
        Custom prompt - With each question, add a prompt asking for feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}
        KLP - With each question add one or two line takeaways for providing feedback. The takeaways should be related to the question it is supplied with.
        KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
        Always use indian names in the role play, also mention what role the user will be playing while answering the questions in the description.
        Always use name in each question. The role play shall also have element of a other person who will be asking the questions.
        Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        The Question, Custom Prompt, KLP, KLS should be numbered.
        Here the format looks like :
        "Title:",
        "Description:”,
        “Statement:",
        "Question 1:",
        "Prompt 1:",
        "Takeaway 1:" ,
        "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
        'The Question, Prompt, Takeaway, Skills should be numbered.'
        NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
        NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
        NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated.
        NOTE: "Rating" must be included.
        NOTE: Make sure the roleplay is very advanced and tough.
        NOTE: Always use a name in each question. The role play shall also have the element of an other person who will be asking the questions.
        NOTE: Always mention in the context what role the user will be playing the role while answering.
        NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        NOTE: Never miss Title, Description, Statement and other variables.
        NOTE: Do not mention "X" or "Y".
        NOTE: must Follow the OUTPUT Format.
        \n\nAssistant:

      '''
    elif scenario_type == 'role_play_static':
      prompt= """
        \n\nHuman:
        {Information} - %s-
        Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
        Description - Define the situation and the problem. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration. It shall always remain between two specific individuals, as determined by the context provided in the {information}.
        Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
        Title must follow this Format:
            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
            Example format:
                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
        Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the problem. NEVER respond to the questions.
        Custom prompt - With each question, add a prompt asking for feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}
        KLP - With each question add one or two line takeaways for providing feedback. The takeaways should be related to the question it is supplied with.
        KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique throughout all questions. Each question shall have unique skills. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
        Always use indian names in the role play, also mention what role the user will be playing while answering the questions in the description.
        Always use name in each question. The role play shall also have element of a other person who will be asking the questions.
        Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        The Question, Custom Prompt, KLP, KLS should be numbered.
        Here the format looks like :
        "Title:",
        "Description:”,
        “Statement:",
        "Question 1:",
        "Prompt 1:",
        "Takeaway 1:" ,
        "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
        'The Question, Prompt, Takeaway, Skills should be numbered.'
        NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
        NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
        NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated.
        NOTE: "Rating" must be included.
        NOTE: Make sure the roleplay is very advanced and tough.
        NOTE: Always use a name in each question. The role play shall also have the element of an other person who will be asking the questions.
        NOTE: Always mention in the context what role the user will be playing the role while answering.
        NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        NOTE: Never miss Title, Description, Statement and other variables.
        NOTE: Do not mention "X" or "Y".
        NOTE: must Follow the OUTPUT Format.
        \n\nAssistant:
        """
    elif scenario_type == "case_static":
      prompt = """
        \n\nHuman:
        {Information} - %s
        Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
        Description - Define the situation and the problem. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
        Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
        Title must follow this Format:
            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
            Example format:
                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
        Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the situation. NEVER provide a response to the questions.
        Custom prompt - With each question, add a prompt that would ask feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide a feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}
        KLP - With each question add one or two line takeaway for providing feedback. The takeaways should be related to the question it is provided with.
        KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
        Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        Always Use a literary genre to generate the response in high literature.
        Literary genres encompass a wide spectrum of styles and themes, ranging from the imaginative realms of fiction, poetry, drama, and fantasy to the factual landscapes of non-fiction, biography, and autobiography. Mystery, science fiction, romance, historical fiction, and horror delve into specific narrative territories, while thriller, adventure, satire, comedy, tragedy, and epic offer diverse storytelling approaches. Additionally, fables, fairy tales, mythology, and folklore explore cultural narratives and traditions. Genres like dystopian, gothic, bildungsroman (coming-of-age), absurdist, and magical realism push the boundaries of conventional storytelling, while realistic fiction and experimental literature offer unique perspectives on reality and form. Each genre contributes to the rich tapestry of literary expression, offering readers a multitude of worlds and experiences to explore.
        In every response, you must:
        Clearly state your role as X.
        Identify Y as the person asking
        The Question, Custom Prompt, KLP, KLS should be numbered.
        Here the format looks like :
        "Title:",
        "Description:”,
        “Statement:",
        "Question 1:",
        "Prompt 1:",
        "Takeaway 1:" ,
        "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
        'The Question, Prompt, Takeaway, Skills should be numbered.'
        NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
        NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
        NOTE: "Rating" must be included.
        NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated.
        NOTE : Make sure the simulation is very advanced and tough.
        NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        NOTE: Always use suitable literary genre to genre create the response.
        NOTE: Never miss Title, Description, Statement and other variables.
        NOTE: Do not mention "X" or "Y".
        NOTE: must Follow the OUTPUT Format.
        \n\nAssistant:

        """

    elif scenario_type == "interview_static":
      prompt = """
        \n\nHuman:
        {Information} - %s
        Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
        Description - Define the situation and the problem. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
        Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
        Title must follow this Format:
            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
            Example format:
                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
        Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the situation. NEVER provide a response to the questions.
        Custom prompt - With each question, add a prompt that would ask feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide a feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}
        KLP - With each question add one or two line takeaway for providing feedback. The takeaways should be related to the question it is provided with.
        KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
        Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        Always use a interview to generate the response for communication and information gathering.
        An interview is a formal conversation between an interviewer and an interviewee, typically in a professional setting, to assess the interviewee's suitability for a particular role or to gather information. It is a common practice in the corporate world and other professional settings, where employers or hiring managers conduct interviews to evaluate potential candidates for employment.
        In every response, you must:
        Clearly state your role as X.
        Identify Y as the person asking
        The Question, Custom Prompt, KLP, KLS should be numbered.
        Here the format looks like :
        "Title:",
        "Description:”,
        “Statement:",
        "Question 1:",
        "Prompt 1:",
        "Takeaway 1:" ,
        "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
        'The Question, Prompt, Takeaway, Skills should be numbered.'
        NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
        NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
        NOTE: "Rating" must be included.
        NOTE : Make sure the simulation is very advanced and tough.
        NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated.
        NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
        NOTE: Always use interview for communication and information gathering.
        NOTE: Never miss Title, Description, Statement and other variables.
        NOTE: Do not mention "X" or "Y".
        NOTE: must Follow the OUTPUT Format.
        \n\nAssistant:
        """
    elif scenario_type == 'checkin_static':
      prompt = """

          \n\nHuman:
          {Information} - %s
          Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
          Description - Define the situation and the problem. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
          Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description related to the check-in.
          Title must follow this Format:
            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
            Example format:
                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
          Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the situation. NEVER provide a response to the questions.
          Custom prompt - With each question, add a prompt that would ask feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide a feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}
          KLP - With each question add one or two line takeaway for providing feedback. The takeaways should be related to the question it is provided with.
          KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined. And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
          Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
          Always use a check-in to generate the response for communication and information gathering.
          Check-in in a corporate setting refers to the process of employees or participants recording their arrival at the workplace, a meeting, a conference, or any other professional gathering. This practice allows for improved attendance tracking, resource allocation, and streamlined communication within the enterprise.
          In every response, you must:
          Clearly state your role as X.
          Identify Y as the person asking
          The Question, Custom Prompt, KLP, KLS should be numbered.
          Here the format looks like :
          "Title:",
          "Description:”,
          “Statement:",
          "Question 1:",
          "Prompt 1:",
          "Takeaway 1:" ,
          "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
          'The Question, Prompt, Takeaway, Skills should be numbered.'
          NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
          NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable like an check-in.
          . Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
          NOTE: "Rating" must be included.
          NOTE : Make sure the simulation is very advanced and tough.
          NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
          NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated.
          NOTE: Always use check-in for communication and information gathering.
          NOTE: Never miss Title, Description, Statement and other variables.
          NOTE: Do not mention "X" or "Y".
          NOTE: must Follow the OUTPUT Format.
          \n\nAssistant:

          """
    elif scenario_type == "static_hard":
      prompt = '''
      \n\nHuman:
            {Information} -
            %s -
            Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
            Description - Define the situation and the problem, and the problem focuses exclusively on hard skills. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
            Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
            Title must follow this Format:
            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
            Example format:
                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
            Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the situation. NEVER provide a response to the questions.Question shall focus exclusively on hard skills
            Custom prompt - With each question, add a prompt that would ask feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}
            KLP - With each question add one or two line takeaways for providing feedback. The takeaways should be related to the question it is provided with.
            KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill and focus exclusively on hard skills. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
            Always end the description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
            In every response, you must:
            Clearly state your role as X as in (information).
            Identify Y as the person asking as in (information)
            The Question, Custom Prompt, KLP, KLS should be numbered.
            Here the format looks like :
            "Title:",
            "Description:”,
            “Statement:",
            "Question 1:",
            "Prompt 1:",
            "Takeaway 1:" ,
            "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
            'The Question, Prompt, Takeaway, Skills should be numbered.'

            NOTE: Description, questions, and skills should focus exclusively on hard skills.
            NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
            NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
            NOTE: KLS - Always each question shall have a unique skill. Thes skill shall be comma separated. AND shall not repeat from (information).
            NOTE: "Rating" must be included.
            NOTE : Make sure the simulation is very advanced and tough.
            NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description] as in (information). Your intent is to achieve Z.
            NOTE: Never miss Title, Description, Statement and other variables.
            NOTE: must Follow the OUTPUT Format.
            \n\nAssistant:


      '''
    elif scenario_type == 'static_soft':
      prompt = """
            (information: %s)

            Carefully review and analyze the provided {information}. Based on this assessment, create a rigorous, high-level simulation that serves as an extended version of the previous scenario, diving deeper into the required skills and interactions. This new scenario must specifically address new areas for candidates to explore, ensuring a targeted approach to tackling an entirely new challenge.

            Key Requirements:
              Create a brand new scenaio in the same industry. Target ONLY soft skills that are not covered in the {information} context.
              Note: Never change the Industry Domain of the scenario.

            Deliver the extended scenario accordingly

            Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
            Description - Define the situation and the problem, and the problem focuses exclusively on soft skills. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
            Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
            Title must follow this Format:
              "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
              Example format:
                    "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
            Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the situation. NEVER provide a response to the questions.Question shall focus exclusively on soft skills
            Custom prompt - With each question, add a prompt that would ask feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}
            KLP - With each question add one or two line takeaways for providing feedback. The takeaways should be related to the question it is provided with.
            KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill and focus exclusively on soft skills. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
            Always end the description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
            In every response, you must:
            Clearly state your role as X as in (information).
            Identify Y as the person asking as in (information)
            The Question, Custom Prompt, KLP, KLS should be numbered.
            Here the format looks like :
            "Title:",
            "Description:”,
            “Statement:",
            "Question 1:",
            "Prompt 1:",
            "Takeaway 1:" ,
            "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
            'The Question, Prompt, Takeaway, Skills should be numbered.'

            NOTE: Description, questions, and skills should focus exclusively on soft skills.
            NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
            NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
            NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated. AND shall not repeat from (SKILLS) instead use different skills.
            NOTE: "Rating" must be included.
            NOTE : Make sure the simulation is very advanced and tough.
            NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description] as in (information). Your intent is to achieve Z.
            NOTE: Never miss Title, Description, Statement and other variables.
            NOTE: Always use the same targeted skill(s) from (targeted_Skills), if available. Fill the skill_list or KLS as applicable using these (targeted skills). Generate every scenario strictly using only this skill(s).
            NOTE: must Follow the OUTPUT Format.
            \n\nAssistant:

      """
    elif scenario_type == 'static_role_play_soft':
      prompt = """
      \n\nHuman:
                {Information} - %s-
                Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
                Description - Define the situation and the problem, and the problem focuses exclusively on soft skills. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration. It shall always remain between two specific individuals, as determined by the context provided in the {information}.
                Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.

                Title must follow this Format:
                  "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
                  Example format:
                        "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
                Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the problem. NEVER respond to the questions. Question shall focus exclusively on soft skills
                Custom prompt - With each question, add a prompt asking for feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}
                KLP - With each question add one or two line takeaways for providing feedback. The takeaways should be related to the question it is supplied with.
                KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill and focus exclusively on soft skills. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
                Always use indian names in the role play, also mention what role the user will be playing while answering the questions in the description.
                Always use name in each question. The role play shall also have element of a other person who will be asking the questions.
                Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
                The Question, Custom Prompt, KLP, KLS should be numbered.
                Here the format looks like :
                "Title:",
                "Description:”,
                “Statement:",
                "Question 1:",
                "Prompt 1:",
                "Takeaway 1:" ,
                "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
                'The Question, Prompt, Takeaway, Skills should be numbered.'

                NOTE: Description, questions, and skills should focus exclusively on soft skills.
                NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
                NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
                NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated.
                NOTE: "Rating" must be included.
                NOTE: Make sure the roleplay is very advanced and tough.
                NOTE: Always use a name in each question. The role play shall also have the element of an other person who will be asking the questions.
                NOTE: Always mention in the context what role the user will be playing the role while answering.
                NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
                NOTE: Never miss Title, Description, Statement and other variables.
                NOTE: Always use the same targeted skill(s) from (targeted_Skills), if available. Fill the skill_list or KLS as applicable using these (targeted skills). Generate every scenario strictly using only this skill(s).
                NOTE: must Follow the OUTPUT Format.
                \n\nAssistant:

      """

    elif scenario_type == 'static_role_play_hard':
      prompt = """
      \n\nHuman:
                {Information} - %s-
                Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
                Description - Define the situation and the problem, and the problem focuses exclusively on hard skills. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration. It shall always remain between two specific individuals, as determined by the context provided in the {information}.
                Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
                Title must follow this Format:
                  "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
                  Example format:
                        "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
                Questions - Develop a set of {%s} question(s) ONLY based on the situation. The questions should be related to the problem. NEVER respond to the questions. Question shall focus exclusively on hard skills
                Custom prompt - With each question, add a prompt asking for feedback from Anthropic about the RESPONSE quality based on best practices. The prompt should ONLY evaluate the quality of the response. NEVER give the prompts to evaluate the questions. Example - {Please provide feedback on the manager's response if the manager focuses on making the team member understand the metrics instead of focusing on the results.}

                KLP - With each question add one or two line takeaways for providing feedback. The takeaways should be related to the question it is supplied with.
                KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined. And For every question choose exactly {%s} skill(s) and not more or less than {%s} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skil and focus exclusively on hard skills. Skill must be only maixmum two words. Each question must have a completely unique set of skills — no skill should repeat in any other question.
                Always use indian names in the role play, also mention what role the user will be playing while answering the questions in the description.
                Always use name in each question. The role play shall also have element of a other person who will be asking the questions.
                Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
                The Question, Custom Prompt, KLP, KLS should be numbered.
                Here the format looks like :
                "Title:",
                "Description:”,
                “Statement:",
                "Question 1:",
                "Prompt 1:",
                "Takeaway 1:" ,
                "Skills 1:" repeated for {%s} question(s). Do not include any {responder} response. NOTE: If {targated skills} are present then skills must be from the {targated skills}.
                'The Question, Prompt, Takeaway, Skills should be numbered.'

                NOTE: Description, questions, and skills should focus exclusively on hard skills.
                NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.
                NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - for example: "Rating : 6". Rating Must be in output. Do not include any other explanation.
                NOTE: KLS - Always each question shall have a unique skill. The skill shall be comma separated.
                NOTE: "Rating" must be included.
                NOTE: Make sure the roleplay is very advanced and tough.
                NOTE: Always use a name in each question. The role play shall also have the element of an other person who will be asking the questions.
                NOTE: Always mention in the context what role the user will be playing the role while answering.
                NOTE: Never miss this, Always end description with this approach and mention this in the “statement”: You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve Z.
                NOTE: Never miss Title, Description, Statement and other variables.

                \n\nAssistant:



      """
    elif scenario_type == 'dynamic_start_with_user':
      prompt = """
      \n\nHuman:
      {Information} - %s

      Read the provided *{information}* thoroughly. Make sure to note the "start-with-user" field, which determines the direction of the conversation. For example, if it shows "customer-sales", then the customer is the responder and the salesperson is the asker. If it shows "sales-customer", then the salesperson is the responder and the customer is the asker. If it shows "team-manager", the team member is the responder and the manager is the asker. If it shows "manager-team", the manager is the responder and the team member is the asker.
      Use this information to construct a realistic, advanced, and specific conversation scenario following the instructions below.
      Description:
      Create a situation that involves a live conversation between two individuals, based on the context given in the *{information}*. The narrative must be written entirely in third person and should clearly state the their roles. Define the situation and the problem. The problem must be directly relevant to the {information}.
      Include rich, specific details such as the setting, context, goals, and underlying tensions. The scenario must stem from a live, real-time conversation and must not involve emails, written exchanges, or hypothetical outcomes. The final output should be between 100 and 200 words, focusing only on the situation and the issue at hand. Do not include dialogue, conclusions, or resolutions.
      The scenario must remain open-ended to allow room for further exploration. All terms and context must be adapted specifically to the {information} provided, ensuring full alignment with the described roles and situation.
      According to the description always end description with this approach and mention this : You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve the {intent}.
      In every response, you must:
      Clearly state your role as X.
      Identify Y as the person asking
      Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
      Title must follow this Format:
            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
            Example format:
                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"

      Prompts - As given in the output format.


      Here the format looks like :
      {
        "Title": "GIVE TITLE",
        "Context": "GIVE DESCRIPTION",
        "Candidate Type": based on information who will respond” ,
        "Scenario Case": "dynamic_discussion",
        "Email Address List": "mail@coachbots.com",
        "Certificate Title": "SAME AS TITLE",
        "Area/Domain": "BASED ON TITLE AND DESCRIPTION",
        "start with user": "based on information",
        "skill_list": "add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {8} skill(s) and not more or less than {8} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. A comma seprated list of skills should be provided. Skill must be only maixmum two words.",
        "is_dynamic_thread": true,
        "Responder": "the second person name who will ask the questions",
        "Person 0": "the second person name who will ask the questions :",
        "0": "Please respond in order to continue.",
        "1": "Now the second person name who will ask the questions will respond to this remark as a Selfish type of person.",
        "2": "Please respond in order to continue.",
        "3": "Now the second person name who will ask the questions will respond to this remark as a Insincere type of person.",
        "4": "Conclude the discussion as a participant.",
      }


      Do not include any response.
      Always provide the output in the given format.

      NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.

      NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - "Rating : 6". Do not include any other explanation.

      NOTE : Make sure the situation is very advanced and tough.

      NOTE : there must be only one manager in picture.

      NOTE : Never miss the Title, Description, Questions.
      NOTE: Do not mention literal "X", "Y".
      NOTE: If {targated skills} are present then "skill_list" must be from the {targated skills}.

      \n\nAssistant:

      """


      return f"{prompt}"%(information)

    if scenario_type == 'normal_dynamic_test':
        prompt = '''
            \n\nHuman:
                          {Information} - (%s)

                          Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:

                          Description - Define the situation and the problem. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.
                          Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
                          According to the description always end description with this approach and mention this : You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve the {intent}.
                          In every response, you must:
                          Clearly state your role as X.
                          Identify Y as the person asking
                          Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
                          Title must follow this Format:
                            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
                            Example format:
                                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
                          Questions - Give me the first question based on the situation in description .The question should be deep, thoughtful and realistic. Give the name of person asking the question. Keep it less than 35 words. NEVER provide a response to the question. Never start with any introduction sentences. Start with the question directly.
                          use this template strictly to generate Questions: ""Thank for connecting. I am looking forward to learning more about the {intent}."". Strictly follow this template structure and do not print any sentence with a question mark.

                          KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {8} skill(s) and not more or less than {8} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. A comma seprated list of skills should be provided. Skill must be only maixmum two words.
                          Output format - Y: question?
                          For example - Ajay: question?

                          Prompts - As given in the output format.

                          Here the format looks like :

                          Title:
                          Description:
                          Questions:
                          Skills:
                          Prompts: - ["Please respond in order to continue.",
                          "Respond as {Y}",
                          ]


                          Note: Add in prompts set of above ["Please respond in order to continue.",
                          "Respond as {Y}",
                          ] for {%s} and at last add "Conclude the discussion as a participant."

                          Write the manager's name in place of {Y}. The Y should always be the same. Do not make any changes in the given format. .

                          Do not include any response.
                          Always provide the output in the given format.
                          NOTE: If {targated skills} are present then "Skills" must be from the {targated skills}.

                          NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.

                          NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - "Rating : 6". Do not include any other explanation.

                          NOTE : Make sure the situation is very advanced and tough.

                          NOTE : there must be only one manager in picture.

                          NOTE : Never miss the Title, Description, Questions, Skills.
                          NOTE: Do not mention literal "X", "Y".
                          NOTE: must Follow the OUTPUT Format.

                          \n\nAssistant:



          '''

        return f"{prompt}"%(information,question_count*2-2)

    if scenario_type == 'normal_dynamic_test_hard':
        prompt = '''
          \n\nHuman:
                        {Information} - (%s)

                        Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
                        Description - Define the situation and the problem and the problem focuses exclusively on hard skills. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
                        According to the description always end description with this approach and mention this : You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve the {intent}.
                        In every response, you must:
                        Clearly state your role as X.
                        Identify Y as the person asking
                        Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
                        Title must follow this Format:
                          "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
                          Example format:
                                "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
                        Questions - Give me the first question based on the situation in description .The question should be deep, thoughtful and realistic. Give the name of person asking the question. Keep it less than 35 words. NEVER provide a response to the question. Question shall focus exclusively on hard skills. Never start with any introduction sentences. Start with the question directly.
                        use this template strictly to generate Questions: ""Thank for connecting. I am looking forward to learning more about the {intent}."". Strictly follow this template structure and do not print any sentence with a question mark.

                        KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {8} skill(s) and not more or less than {8} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. and focus exclusively on hard skills. A comma seprated list of skills should be provided. Skill must be only maixmum two words.
                        Output format - Y: question?
                        For example - Ajay: question?

                        Prompts - As given in the output format.

                        Here the format looks like :

                        Title:
                        Description:
                        Questions:
                        Skills:
                        Prompts: - ["Please respond in order to continue.",
                        "Respond as {Y}",
                        ]


                        Note: Add in prompts set of above ["Please respond in order to continue.",
                        "Respond as {Y}",
                        ] for {%s} and at last add "Conclude the discussion as a participant."

                        Write the manager's name in place of {Y}. The Y should always be the same. Do not make any changes in the given format. .

                        Do not include any response.
                        Always provide the output in the given format.
                        NOTE: Description, questions, and Skills should focus exclusively on hard skills.
                        NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.

                        NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - "Rating : 6". Do not include any other explanation.

                        NOTE : Make sure the situation is very advanced and tough.

                        NOTE : there must be only one manager in picture.

                        NOTE : Never miss the Title, Description, Questions, Skills.
                        NOTE: Do not mention literal "X", "Y".
                        NOTE: If {targated skills} are present then "Skills" must be from the {targated skills}.
                        NOTE: must Follow the OUTPUT Format.

                        \n\nAssistant:


        '''

        return f"{prompt}"%(information,question_count*2-2)

    if scenario_type == 'normal_dynamic_test_soft':
        prompt = '''
                        \n\nHuman:
                        {Information} - (%s)

                        Read this {information} thoroughly. Now, based on this information and your understanding, create an advanced and detailed scenario to practice the skills related to {skill_domain}. The scenario should be in the {department} department of a {industry} company. After creating the situation, provide these:
                        Description - Define the situation and the problem and the problem focuses exclusively on soft skills. The problem should be related to the {information} provided. The description should always involve a conversation based on the context of the {information}. Make the description specific based on the information, focusing on relevant details. Provide the name of the individual in a leadership role based on the information, but never provide the name of the other person involved. The description should detail the situation that led to the issue, and it should not include dialogues. It should always be from a third-person perspective. Provide the description in 100 to 200 words. Avoid adding any conclusions. The situation should not be related to writing emails.        Always replace the terms with those relevant to the {information}, ensuring the scenario is based on the specifics of the context and is open-ended for further exploration.
                        According to the description always end description with this approach and mention this : You are X, interacting with Y. Y will ask you questions related to [description]. Your intent is to achieve the {intent}.
                        In every response, you must:
                        Clearly state your role as X.
                        Identify Y as the person asking
                        Title - Give a specific and relevant title for this description. The title should NEVER be less than 8 words. The title should always be directly related to the given description. Make it very specific to the description.
                        Title must follow this Format:
                            "[Short 3-4 word core areas] ( High Level Skill : skill1, skill2)"
                            Example format:
                                  "AI-Powered E-commerce Chatbots (High-Level Skill: NLP, Dialogflow)"
                        Questions - Give me the first question based on the situation in description .The question should be deep, thoughtful and realistic. Give the name of person asking the question. Keep it less than 35 words. NEVER provide a response to the question. Question shall focus exclusively on soft skills. Never start with any introduction sentences. Start with the question directly.
                        use this template strictly to generate Questions: ""Thank for connecting. I am looking forward to learning more about the {intent}."". Strictly follow this template structure and do not print any sentence with a question mark.

                        KLS - With each question, add the skill(s) that are highlly relevant to the scenario. All skills should be from the {skill_domain} only and must be taken from {targated skills} if defined.  And For every question choose exactly {8} skill(s) and not more or less than {8} should be chosen for each question. The skills for all the questions should be unique. Each question shall have a unique skill. and focus exclusively on soft skills. A comma seprated list of skills should be provided. Skill must be only maixmum two words.
                        Output format - Y: question?
                        For example - Ajay: question?

                        Prompts - As given in the output format.skill_list

                        Here the format looks like :

                        Title:
                        Description:
                        Questions:
                        Skills:
                        Prompts: - ["Please respond in order to continue.",
                        "Respond as {Y}",
                        ]


                        Note: Add in prompts set of above ["Please respond in order to continue.",
                        "Respond as {Y}",
                        ] for {%s} and at last add "Conclude the discussion as a participant."

                        Write the manager's name in place of {Y}. The Y should always be the same. Do not make any changes in the given format. .

                        Do not include any response.
                        Always provide the output in the given format.
                        NOTE: Description, questions, and Skills should focus exclusively on soft skills.
                        NOTE: The title should NEVER be less than 8 words. Make the title detailed for the description.

                        NOTE : Based on this information {information} please evaluate this scenario provides a good practice to improve the skills that are given in the scenario. Evaluate whether the scenario is relevant and understandable. Give the scenario an overall rating out of 10. Just give the rating in the output in this format - "Rating : 6". Do not include any other explanation.

                        NOTE : Make sure the situation is very advanced and tough.

                        NOTE : there must be only one manager in picture.

                        NOTE : Never miss the Title, Description, Questions, Skills.
                        NOTE: Do not mention literal "X", "Y".
                        NOTE: If {targated skills} are present then "Skills" must be from the {targated skills}.
                        NOTE: must Follow the OUTPUT Format.
                        \n\nAssistant:


        '''

        return f"{prompt}"%(information,question_count*2-2)


    return f"{prompt}"%(information, question_count, skill_count, skill_count, question_count)



from string import Template


def get_game_prompt(industry, information, num_of_questions, question_type, candidate_type):
  prompt = '''
    Create a large "${Industry}" corporate scenario is less than 100 words with a title upto 8-12 words that related to : (${information}).

    Further create ${num_of_questions} MCQ questions with 4 options each that are related to the paragraph that the user must answer as a new manager tasked with solving the issue at hand.
    The Questions must have 4 options and will have ${question_type} right answer - however they should not be straightforward and it may appear other choices are right as well.
    Always end the description with As a ${candidate_type} select the right option for the questions presented below.

    GIVE IN THIS VALID JSON FORMAT:
    json ```
    {
      "title": "title goes here",
      "description": "decrtiption title goes here",
      "is_single_select": "TRUE or FALSE",
      "questions" :[
        {
          ""context"": {
            ""section"": ""Section Text""
          },
          ""details"": {
            ""question"": ""Question Text""
          },
          ""content"": {
            ""instruction"": ""Choose one or more options from A, B, C or D"",
            ""options"": {

                ""A"": ""Option A"",
                ""B"": ""Option B"",
                ""C"": ""Option C"",
                ""D"": ""Option D""

            }
          }
        },
      ],
    }
    ```

    (If the prompt mentions "single," then the value of "is_single_select" should be "TRUE" instead.)
    (If the prompt mentions "multiple," then the value of "is_single_select" should be "FALSE" instead.)
    NOTE: All keys required in output format.

    '''

  return Template(prompt).substitute(Industry=industry, information=information, num_of_questions=num_of_questions, question_type=question_type, candidate_type=candidate_type)

def format_game_custom_prompt(is_single_select, questions, title, description, num_of_questions=None, static=True):
  instruction = "Choose one option from A, B, C or D" if is_single_select else "Choose one or more options from A, B, C or D"
  if static:
    questions = "\n\n".join([str(i) for i in questions])
    custom_prompt = """
    **Prompt Guidelines:**

    1. **Display the End Game Message**: Ensure the final message appears as specified, substituting 'x' with the player's total score and replacing '[Game Name]' with the actual game title:
      - Congratulations 🎉. You have completed the [Game Name]. You have achieved a score of [x out of 100].

    2. **No clipping or trucation of text**: Ensure that each option is presented in its entirety, without any clipping or truncation of text. Do not hallucinate or invent options; present only the options exactly as provided in the game design.

    3. **Demand Correct Input for Progression**: Require players to input a valid choice precisely to advance to subsequent levels. Repeat the prompt until a correct input is received.

    4. **Display the Feedback**: Upon game completion, provide approximately 50 words of feedback summarizing the impact of the user's choices on the outcome. Briefly suggest alternative strategies for potentially improved results in future playthroughs.

    5. **Always Present Full Level Details**: Consistently show all levels and options exactly as scripted. Complete all levels and only give feedback as last. Every level and option must be displayed in full, without omission or partial rendering of any text.

    6. **Start with Level Prompt**: When the command """"START"""" is given, begin immediately with the first level.

    7. **Output in JSON format**:  Ensure that the game’s output is formatted as valid JSON. Below is the structure:
    json
    if game not ended:
      ``` json
      {
        ""context"": {
          ""section"": ""Section Text""
        },
        ""details"": {
          ""question"": ""Question Text""
        },
        ""content"": {
          ""instruction"": ""${instruction}"",
          ""options"": {

              ""A"": ""Option A"",
              ""B"": ""Option B"",
              ""C"": ""Option C"",
              ""D"": ""Option D""

          }
        }
      }
    ```
    if game ended:
    json
    {
    ""end_message"" : ""[End Game Message]"",
    ""feedback"": ""[Feedback Text]""
    }


    Let's continue with the game using these guidelines.

    ---
    ## Title:
    ${title}

    ## Overview & Gameplay Objectives:
    ${description}

    ---
    if game not ended:
    Ensure that the game’s output is formatted as valid JSON. Below is the structure:
    json
    if game not ended:
    ```json
    ${questions}
    ```
    ---

    ## End Game Message:
    Congratulations 🎉. You have completed the [Game Name]. You have achieved a score of [x out of 100].

    ## Feedback:
    Provide 50 words of feedback regarding the answers of the options chosen by the user, and suggest if they could have done anything better."
    """

    return Template(custom_prompt).substitute(instruction=instruction,questions=questions,title=title,description=description)
  else:
    custom_prompt = """
    **Prompt Guidelines:**

    1. **Display the End Game Message**: Ensure the final message appears as specified, substituting 'x' with the player's total score and replacing '[Game Name]' with the actual game title:
      - Congratulations 🎉. You have completed the [Game Name]. You have achieved a score of [x out of 100].

    2. **No clipping or trucation of text**: Ensure that each option is presented in its entirety, without any clipping or truncation of text. Do not hallucinate or invent options; present only the options exactly as provided in the game design.

    3. **Demand Correct Input for Progression**: Require players to input a valid choice precisely to advance to subsequent levels. Repeat the prompt until a correct input is received.

    4. **Display the Feedback**: Upon game completion, provide approximately 50 words of feedback summarizing the impact of the user's choices on the outcome. Briefly suggest alternative strategies for potentially improved results in future playthroughs.

    5. **Always Present Full Level Details**: Consistently show all levels and options exactly as scripted. Complete all levels and only give feedback as last. Every level and option must be displayed in full, without omission or partial rendering of any text.

    6. **Start with Level Prompt**: When the command """"START"""" is given, begin immediately with the first level.

    7. **Output in JSON format**:  Ensure that the game’s output is formatted as valid JSON. Below is the structure:
    json
    if game not ended:
      ``` json
    {
        ""context"": {
          ""section"": ""Section Text""
        },
        ""details"": {
          ""question"": ""Question Text""
        },
        ""content"": {
          ""instruction"": ""${instruction}"",
          ""options"": {

              ""A"": ""Option A"",
              ""B"": ""Option B"",
              ""C"": ""Option C"",
              ""D"": ""Option D""

          }questions
        }
      }
    ```
    if game ended:
    json
    {
    ""end_message"" : ""[End Game Message]"",
    ""feedback"": ""[Feedback Text]""
    }

    8. **Craft Challenging Decision Options**: Offer related and nuanced options prompting strategic contemplation for informed decision-making.

    9. **Total Number of Levels** = ${number_of_level}

    Let's continue with the game using these guidelines.

    ---

    "## Title:
    ""${title}""
    ## Overview & Gameplay Objectives:
    ${description}
    ---
    if game not ended:
    Ensure that the game’s output is formatted as valid JSON. Below is the structure:
    json
    if game not ended:
      ```json
    {
        ""context"": {
          ""section"": ""Section Text""
        },
        ""details"": {
          ""question"": ""Question Text""
        },
        ""content"": {
          ""instruction"": ""${instruction}"",
          ""options"": {

              ""A"": ""Option A"",
              ""B"": ""Option B"",
              ""C"": ""Option C"",
              ""D"": ""Option D""

          }
        }
      }
```
    ---

    ## End Game Message:
    Congratulations 🎉. You have completed the [Game Name]. You have achieved a score of [x out of 100].

    ## Feedback:
    Provide 50 words of feedback regarding the answers of the options chosen by the user, and suggest if they could have done anything better.


        """

    return Template(custom_prompt).substitute(instruction=instruction,title=title,description=description,number_of_level=num_of_questions)


def feedback_video_script_template(responder, asker,department, industry, objective, domain_skill, klps, dynamic=False):
    script = f"""
        Hey, This is Maya. your feedback coach. Thank you for completing your roleplay simulation. Putting yourself in the shoes of the {responder} in the {department} Department of the {industry} can be challenging! When you are dealing with such situations with your {asker} do this.  Always take a pause. See what are the key skills you must possess. And how you should demonstrate them. In other words - what to say. And how to say it. To unpack it further here are the basics: {objective}. Your level II skill details you can find in the report.
    """
    klps_script = f"""
    The key learning points here : "
    {klps} ".
    Don't forget to check out the details in the report!"""

    if dynamic:
      klps_script = """
        The key learning points here : "${klps} ".
      Don't forget to check out the details in the report!
      """
    return script + "\n" + klps_script


###Functions initialization

In [ ]:
def clean_text(input_text):
    # Remove all types of brackets and double quotes
    cleaned_text = re.sub(r'[\[\]\(\)\{\}<>"]', '', input_text)
    cleaned_text = cleaned_text.strip().replace('"',"").replace("'","").replace('[','').replace(']','').replace('{}','').replace('}','').replace('(','').replace(')','')
    return cleaned_text

def remove_url_garbage_char(text):
    # Remove URLs
    text = clean_text(text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove common encoding garbage characters
    garbage_chars = [
        'â€“', 'â€™', 'â€œ', 'â€', 'â€˜', 'â€', 'â€¢', 'Â', '€', '™', '�'
    ]
    for char in garbage_chars:
        text = text.replace(char, '')

    # Optionally remove any leftover non-ASCII characters
    text = text.encode('ascii', 'ignore').decode('ascii')

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def generate_video_script_via(objective, prompt= None):
  if not prompt:
    prompt = video_script_prompt(objective)
  print('prompt', prompt)
  result = gemini_completion(prompt)

  result = remove_url_garbage_char(result)
  return result

def create_csv(scenario_data,question_count,iteration,scenario_type,information,file_name=None):
      try:
        print('create_csv', scenario_data)
        response = {}

        response['Error'] = scenario_data.get('error')
        response['Intake Information'] = information
        response['Scenario Prompt Type'] = scenario_type
        response['Title'] = scenario_data.get('title')
        response['Test description'] = scenario_data.get('description')


        # Add extra columns
        const_columns = "Skill Domain,Candidate Type,Email Address List,Interaction Mode,Test Type,Scenario Case,Area/Domain,Certificate Title,Client Name,Personality Model,"

        optional = "Script Video Link,Video Script,Feedback Video Link,Feedback Video Script,Test Snippet Link,Max Test Allowed,Description Media,is learner path,Ted talks and HBR Case,is checkin type,is_email_type,Send only to email,Email Candidate,Certificate Description,source,image_url,rating,is_game_type,Competency Skill,Goals,Course,Industry,Experience Level,Background,Title UI,Description UI,is_micro,is_logged_in,Is Immersive,Is Transcript Only,is_pitch,Current news,Bot Name,User ID,Tab Category,Visual Tags,Page Name,"

        # if scenario_data.get('personality_model', None):
        #   const_columns += "Personality Model,"

        const_columns_list = const_columns.split(',')

        for col_name in const_columns_list:
            if len(col_name) == 0:
              continue
            response[col_name] = None

        response['Certificate Title'] = scenario_data.get('title')
        response['Email Address List'] = 'mail@coachbots.com'
        response['Interaction Mode'] = 'any'
        response['Test Type'] = 'test'
        response['Scenario Case'] = "role_play" if 'role_play' in scenario_type  else "simulation"
        response['Area/Domain'] = 'General'


        if scenario_data.get('personality_model', None):
          print(scenario_data.get('personality_model'),'personality_model')
          response['Personality Model'] = scenario_data.get('personality_model')


        if scenario_data.get('skill_domain'):
          response['Skill Domain'] = scenario_data.get('skill_domain')

        if scenario_data.get('candidate_type'):
          response['Candidate Type'] = scenario_data.get('candidate_type')


        print('Title: ', response['Title'])
        print('Test description: ', response['Test description'])

        questions = [que.get('question') for que in scenario_data.get('question_info')]
        prompts = [que.get('gpt_prompt_override') for que in scenario_data.get('question_info')]
        takeaways = [que.get('key_learning_point') for que in scenario_data.get('question_info')]
        skills_match = [que.get('key_learning_skills') for que in scenario_data.get('question_info')]

        if len(questions) > question_count:
          questions = questions[:question_count]

        if len(prompts) > question_count:
          prompts = prompts[:question_count]

        if len(takeaways) > question_count:
          takeaways = takeaways[:question_count]

        if len(skills_match) > question_count:
          skills_match = skills_match[:question_count]

        print('questions :', questions)
        print('prompts :', prompts)
        print('takeaways :', takeaways)
        print('skills_match :', skills_match)

        # Store the extracted information in the response dictionary
        for i in range(len(questions)):
            response[f'Question {i+1}'] = questions[i]
            response[f'Custom Prompt {i+1}'] = prompts[i]
            response[f'KLP {i+1}'] = takeaways[i]
            try:
                response[f'KLS {i+1}'] = skills_match[i].replace(' and', ',')
            except:
                response[f'KLS {i+1}'] = ""





        # adding optional fields
        for opt in optional.split(','):
          if len(opt) == 0:
              continue
          response[opt] = None

        print('video_script', scenario_data.get('video_script'))
        if scenario_data.get('video_script',None):
          print('video_script2', scenario_data.get('video_script'))

          response['Video Script'] = scenario_data.get('video_script')
        if scenario_data.get('script_video_link'):
          response['Script Video Link'] = scenario_data.get('script_video_link')
        if scenario_data.get('feedback_video_link'):
          response['Feedback Video Link'] = scenario_data.get('feedback_video_link')
        if scenario_data.get('feedback_script'):
          response['Feedback Video Script'] = scenario_data.get('feedback_script')


        print("REsponse to create csv", response)
        df = pd.DataFrame([response])
        file_path =file_name if file_name else f'bulk_static_{question_count}_que-{file_name}.csv'
        my_string = ""
        if not os.path.isfile(file_path):

            with open(file_path, 'a') as file:
                my_string = 'Error,Intake Information,Scenario Prompt Type,Title,Test description,' + const_columns
                for cnt in range(1, question_count+1):
                  my_string += f"Question {cnt},Custom Prompt {cnt},KLP {cnt},KLS {cnt},"

                my_string += optional
                my_string = my_string[:len(my_string)-1] + "\n"

                file.write(my_string)
                my_string = my_string

        else:
            with open(file_path, 'r') as file:
                my_string = file.read().split('\n')[0]

        print(df.columns)
        print(len(df.columns))
        print('&'*100)

        print(my_string)
        print(len(my_string.split(',')))
        print('*'*100)
        if (len(df.columns) == (len(my_string.split(',')))):
            df.to_csv(file_path, mode='a', index=False, header=False)
            print(f'Saved {iteration} scenarios in the csv.\n\n')
        else:
            print(f'Error in iterations {iteration}. could not save to csv')
            print("Error: Number of columns in the row doesn't match the table.\n\n")
      except Exception as e:
        print('*'*100)
        print(f'[Got Error Skipping this test]: {e}')
        raise e



## extracter
import re

def extract_text_only(input_text):
    # Remove digits from the text
    text_without_digits = re.sub(r'\d', '', input_text)

    # Remove extra whitespaces
    cleaned_text = ' '.join([st.replace("-","").strip().capitalize()  for st in text_without_digits.replace("."," ").strip().split()])

    return cleaned_text



def extract_information_static(text,iteration,question_count,scenario_type,candidate_type,is_pitch=False,personality_model=None,intake={},generate_video_script=False,video_script=None):
    """
    Extract information from a given text containing details about a scenario.

    Parameters:
    - text (str): The text containing information about a scenario.

    Returns:
    - tuple: A tuple containing title, description, question_info, skill_to_evaluate, and rating.

    Example:
    >>> extract_information('Title: Test\nDescription: Test Description\nQuestion: What is your approach to leadership?\nPrompt: Provide your leadership style.\nTakeaway: Effective communication is key.\nSkills: Communication, Leadership\nRating: 5')
    # Returns a tuple with extracted information from the scenario text.
    """
    # Regular expressions for extracting title, description, questions, prompts, takeaways, and skills
    try:
      text = text.replace("KLS", "Skills")
      text = text.replace("KLP", "Takeaway")
      text = text.replace("Custom prompt", "Prompt")
      text = text.replace("*","")

      title_pattern = re.compile(r'Title\s*:\s*(.+)')
      description_pattern = re.compile(r'Description\s*:\s*(.+)')
      statement_pattern = re.compile(r'Statement\s*:\s*(.+)')
      # background_pattern = re.compile(r'Background\s*:\s*(.+)')

      question_pattern = re.compile(r'Question\s*(\d*)\s*:\s*(.+)')
      prompt_pattern = re.compile(r'Prompt\s*(\d*)\s*:\s*(.+)')
      takeaway_pattern = re.compile(r'Takeaway\s*(\d*)\s*:\s*(.+)')
      skills_pattern = re.compile(r'Skills\s*(\d*)\s*:\s*(.+)')
      rating_pattern = re.compile(r'Rating\s*:\s*(\d+)')

      # Extracting information using regular expressions
      title_match = title_pattern.search(text)
      description_match = description_pattern.search(text)
      rating_match = rating_pattern.search(text)
      statement_match = statement_pattern.search(text)
      # background_match = background_pattern.search(text)




      # If title_pattern doesn't match, try to find the title as the first line before the description
      if not title_match:
          pattern = re.compile(r'^(?:Title\s*:\s*)?(?:"(.*?)"|([^"\n]*))\n*Description\s*:')
          title_match = pattern.search(text)
          if not title_match:
              # Extract title (first quoted string or first line before description)
              title_match = re.search(r'^"([^"]+)"', text)
              title = title_match.group(1) if title_match else None
              if not title:
                raise ValueError("Invalid format. Unable to extract the title.")

      if not (title_match and statement_match and description_match and  question_pattern.findall(text) and prompt_pattern.findall(text) and takeaway_pattern.findall(text) and skills_pattern.findall(text)):
          invalid_fields = []

          if not title_match:
              invalid_fields.append("title")

          if not description_match:
              invalid_fields.append("description")
          if not question_pattern.findall(text):
              invalid_fields.append("question pattern")
          if not prompt_pattern.findall(text):
              invalid_fields.append("prompt pattern")
          if not takeaway_pattern.findall(text):
              invalid_fields.append("takeaway pattern")
          if not skills_pattern.findall(text):
              invalid_fields.append("skills pattern")
          if not statement_match:
              invalid_fields.append("statement pattern")
          # if not background_match:
          #     invalid_fields.append("background pattern")

          raise ValueError(f"Invalid format. Unable to extract necessary information. Invalid fields: {', '.join(invalid_fields)}")

      title = title_match.group(1) if title_match.group(1) else title_match.group()
      description = f'{clean_text(description_match.group(1))}'
      if statement_match:
          description += f"\nStatement: {clean_text(statement_match.group(1))}"
      # if background_match:
      #     description += f'\nBackground: {background_match.group(1)}'

      rating = int(rating_match.group(1)) if rating_match else 0
      questions = []
      for match in question_pattern.finditer(text):
          question_number = match.group(1) if match.group(1) else len(questions) + 1
          question_text = clean_text(match.group(2))
          prompt_match = prompt_pattern.search(text, match.end())
          takeaway_match = takeaway_pattern.search(text, prompt_match.end())
          skills_match = skills_pattern.search(text, takeaway_match.end())
          print(skills_match)

          prompt_text = clean_text(prompt_match.group(2))
          takeaway_text = clean_text(takeaway_match.group(2))
          skills_text = clean_text(skills_match.group(2))
          question_data = {
              'text': question_text,
              'prompt': prompt_text,
              'takeaway': takeaway_text,
              'skills': skills_text
          }
          questions.append(question_data)

      informations = {
          'title': title,
          'description': description,
          'rating': rating,
          'questions': questions
      }

      title = informations['title']

      question_info = []
      skill_to_evaluate = set()
      klps = []

      for que in informations['questions']:
          question_info.append({
              "question": que["text"],
              "question_type": "subjective",
              "gpt_prompt_override": clean_text(que["prompt"]),
              "subjective_answer": "",
              "key_learning_point": extract_text_only(que['takeaway']),
              "key_learning_skills": extract_text_only(que['skills'])
          })
          klps.append(extract_text_only(que['takeaway']))

          for skill in que['skills'].split(','):
              skill_to_evaluate.add(extract_text_only(skill.strip().capitalize()))

      if len(skill_to_evaluate) < 6 and not is_pitch:
          raise ValueError(f"Skills must have at least 6. Got:  {len(skill_to_evaluate)}, {skill_to_evaluate}")


      skill_to_evaluate = ', '.join(list(skill_to_evaluate)[:8])

      infomation = {
          'title': title,
          'description': description,
          'question_info': question_info,
          'skill_to_evaluate': skill_to_evaluate,
          'rating': rating,
          'personality_model': personality_model,
          'candidate_type': candidate_type,
          'skill_domain': intake.get('Skill Domain', "")
      }
      feedback_script = feedback_video_script_template(intake.get('Responder'),
                                                      intake.get('Asker'),
                                                      intake.get('Department'),
                                                      intake.get('Industry'),
                                                      intake.get('Objective'),
                                                      intake.get('Domain Skill'),
                                                      '\n\n'.join(klps),
                                                      )
      infomation['feedback_script'] = feedback_script
      if video_script:
          infomation['video_script'] = video_script
      elif generate_video_script:
        infomation['video_script'] = generate_video_script_via(description)

      print(f'scenario info: {infomation}')
      return infomation, True

    except Exception as e:
      question_info = []
      for i in range(question_count):
        question_info.append(
            {
                "question": "Error",
                "question_type": "subjective",
                "gpt_prompt_override": "Error",
                "subjective_answer": "",
                "key_learning_point": "Error",
                "key_learning_skills": "Error"
            }
        )
      return {'error': str(e), 'question_info': question_info}, False



def get_skills(candidate_type):

    MANAGER = [

        "Communication skills",

        "Objection handling",

        "Problem solving",

        "Social skills",

        "Collaboration",

        "Accountable",

        "Improve lives around you",

        "Negotiation",

        "Get the best from others",

        "Flexible",

        "Coaching",

        "Methodical approach",

        "Empathy",

        "Decisiveness",

        "Self assurance",

        "Clarity and concision"

    ]



    SALES_MANAGER = [

        "Communication skills",

        "Presence",

        "Ability to inspire",

        "Persuasive",

        "Strategic thinking",

        "Negotiation",

        "Presentation skills",

        "Problem solving",

        "Methodical approach",

        "Time management",

        "Storytelling",

        "Standards",

        "Tenacity",

        "Patience",

        "Curiosity",

        "Passionate"

    ]



    CUSTOMER_SERVICES = [

        "Communication skills",

        "Presence",

        "Social skills",

        "Coaching",

        "Flexible",

        "Ability to confront others",

        "Collaboration",

        "Ability to pivot",

        "Problem solving",

        "Accountable",

        "Clarity and concision",

        "Focused",

        "Empathy",

        "Proactive",

        "Willingness to learn",

        "Decisiveness"

    ]



    EMPLOYEE = [

        "Communication skills",

        "Objection handling",

        "Problem solving",

        "Social skills",

        "Collaboration",

        "Accountable",

        "Improve lives around you",

        "Negotiation",

        "Get the best from others",

        "Flexible",

        "Coaching",

        "Methodical approach",

        "Empathy",

        "Decisiveness",

        "Self assurance",

        "Clarity and concision"

    ]





    if candidate_type == 'Manager':

      skills_list = MANAGER

    elif candidate_type == 'Sales Manager':

      skills_list = SALES_MANAGER

    elif candidate_type == 'Customer Service':

      skills_list = CUSTOMER_SERVICES

    elif candidate_type == 'Employee':

        skills_list = EMPLOYEE

    else:

      skills_list = []





    skills_to_evalute = set()

    for skills in skills_list:

      skills_to_evalute.add(skills.strip().capitalize())



    return skills_to_evalute


def dynamic_create_csv(scenario_data,question_count,iteration,scenario_type,is_game=False,information=None,file_name=None):
      try:
        response = {}
        response['Error'] = scenario_data.get('error')
        response['Intake Information'] = information
        response['Scenario Prompt Type'] = scenario_type
        response['Title'] = scenario_data.get('title')
        response['Context'] = scenario_data.get('description')

        # Add extra columns
        if is_game:
          const_columns = "Test Custum Prompt,is_dynamic_thread,Email Address List,Scenario Case,Is Single Select,"
          optional = ""
        else:
          const_columns = "Script Video Link,Video Script,Feedback Video Link,Feedback Video Script,Skill Domain,Candidate Type,Email Address List,Scenario Case,Area/Domain,Certificate Title,Client Name,is_dynamic_thread,Asker UI,start with user,Skills_list,"
          # if scenario_data.get('start_with_user') != None:
          #   const_columns += "start with user,"

          print(scenario_data.get('skills_list'), 'skills_list')
          # if scenario_data.get('skills_list'):
          #   const_columns += 'Skills_list,'


          optional = "Test Snippet Link,Max Test Allowed,Description Media,is learner path,Ted talks and HBR Case,is checkin type,is_email_type,Send only to email,Email Candidate,Certificate Description,source,image_url,rating,is_game_type,Competency Skill,Goals,Course,Industry,Experience Level,Background,Title UI,Description UI,is_micro,is_logged_in,Is Immersive,Is Transcript Only,is_pitch,Current news,Bot Name,User ID,Tab Category,Visual Tags,Page Name,"

        const_columns_list = const_columns.split(',')

        for col_name in const_columns_list:
            if len(col_name) == 0:
              continue
            response[col_name] = None


        extraa_fields = ""
        for index, initial_question in enumerate(scenario_data.get('orchestrated_conversation_details',{}).get('initial_messages',[])):
          response[f'Person {index}'] = initial_question
          extraa_fields += f'Person {index},'

        for index,question in enumerate(scenario_data.get('question_info', {})):
          response[str(index)] = question.get('question')
          extraa_fields += f'{index},'




        response['Scenario Case'] = "game" if is_game else 'dynamic_discussion'
        response['is_dynamic_thread'] = True
        if is_game:
          print(isinstance(scenario_data.get('game_questions'), list))
          try:
            response['Test Custum Prompt'] = format_game_custom_prompt(is_single_select=scenario_data.get('is_single_select'),
                                                                      questions=scenario_data.get('game_questions'),
                                                                      title=scenario_data.get('title'),
                                                                      description=scenario_data.get('description'),
                                                                      num_of_questions=question_count,
                                                                      static= True if scenario_type == 'static_game' else False
                                                                  )
          except Exception as e:
            print("failed to custom prompt", e)
            raise e

          if scenario_data.get('is_single_select') != None:
            response['Is Single Select'] = scenario_data.get('is_single_select')

        if scenario_data.get('skill_domain'):
          response['Skill Domain'] = scenario_data.get('skill_domain')

        if 'Candidate Type' in response and scenario_data.get('candidate_type'):
          response['Candidate Type'] = scenario_data.get('candidate_type')
        if 'Email Address List' in response and scenario_data.get('email_list'):
          response['Email Address List'] = scenario_data.get('email_list')
        if 'Area/Domain' in response:
          response['Area/Domain'] = scenario_data.get('area_domain') if scenario_data.get('area_domain') else 'General'
        if 'Certificate Title' in response and scenario_data.get('certificate_title'):
          response['Certificate Title'] = scenario_data.get('certificate_title')
        if 'Description Media' in response and scenario_data.get('description_media'):
          response['Description Media'] = scenario_data.get('description_media')
        print("scenario_data.get('start_with_user')", scenario_data.get('start_with_user'))
        if 'start with user' in response and scenario_data.get('start_with_user') != None :
          response['start with user'] = scenario_data.get('start_with_user')
        if 'Asker UI' in response and scenario_data.get('responder'):
          response['Asker UI'] = scenario_data.get('responder')
        if 'Skills_list' in response and scenario_data.get('skills_list'):
          response['Skills_list'] = scenario_data.get('skills_list')

        response['Email Address List'] = 'mail@coachbots.com'



        # adding optional fields
        for opt in optional.split(','):
          if len(opt) == 0:
              continue
          response[opt] = None

        if scenario_data.get('video_script'):
          response['Video Script'] = scenario_data.get('video_script')
        if scenario_data.get('script_video_link'):
          response['Script Video Link'] = scenario_data.get('script_video_link')
        if scenario_data.get('feedback_video_link'):
          response['Feedback Video Link'] = scenario_data.get('feedback_video_link')

        if scenario_data.get('feedback_script'):
          response['Feedback Video Script'] = scenario_data.get('feedback_script')



        df = pd.DataFrame([response])
        file_path = file_name if file_name else (f'bulk_game_{question_count}_que.csv' if is_game else f'bulk_dynamic_{question_count}_que.csv')
        my_string = ""
        if not os.path.isfile(file_path):

            with open(file_path, 'a') as file:
                my_string = 'Error,Intake Information,Scenario Prompt Type,Title,Context,' + const_columns
                my_string += extraa_fields
                my_string += optional
                my_string = my_string[:len(my_string)-1] + "\n"


                file.write(my_string)
                my_string = my_string

        else:
            with open(file_path, 'r') as file:
                my_string = file.read().split('\n')[0]

        print(df.columns)
        print(len(df.columns))
        print('&'*100)

        print(my_string)
        print(len(my_string.split(',')))
        print('='*100)
        if (len(df.columns) == (len(my_string.split(',')))):
            df.to_csv(file_path, mode='a', index=False, header=False)
            print(f'Saved {iteration} scenarios in the csv.\n\n')
        else:
            print(f'Error in iterations {iteration}. could not save to csv')
            print("Error: Number of columns in the row doesn't match the table.\n\n")
      except Exception as e:
        print('*'*100)
        print(f'[Got Error Skipping this test]: {e}')
        raise e

def fetch_json_from_string(text):
    """
    Extracts and parses the first valid JSON object from a given string.

    :param text: str - A string that contains JSON data mixed with other text.
    :return: dict or None - Extracted JSON data or None if parsing fails.
    """
    try:
        # Regular expression to find JSON-like structures
        json_match = re.search(r'\{.*\}', text, re.DOTALL)

        if json_match:
            json_string = json_match.group(0)  # Extract JSON substring
            return json.loads(json_string)  # Parse JSON
        else:
            print("No valid JSON found in the text.")
            return None
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None

def extract_information_dynamic_scenario(text,scenario_type,candidate_type="Manager",num_questions=3,targeted_skills=None,intake={},generate_video_script=False,video_script=None):

    """

    Extract information from a dynamic scenario text.



    Parameters:

    - text (str): The dynamic scenario text to extract information from.

    - is_dynamic (bool): Indicates whether the scenario is dynamic.

    - candidate_type (str): Type of candidate (e.g., 'Manager', 'Team Member').



    Returns:

    - tuple: A tuple containing title, description, question_info, rating, evaluation_skill_list, and orchestrated_conversation_details.



    Example:

    >>> extract_information_dynamic_scenario('Title: Test Title\nDescription: Test Description\nQuestion: What is your approach to leadership?\nRating: 5', is_dynamic=True, candidate_type='Manager')

    # Returns a tuple with extracted information from the dynamic scenario text.

    """

    if not text:
        raise ValueError("Invalid format. Text is empty.")

    try:


      data = fetch_json_from_string(text)


      if data:
        print('data', data)
        question_info = []
        manager_name = data['Person 0'].split(':')[0].strip()
        title = data['Title']
        description = data['Context']

        # for key, value in data.items():
        #   if key.isdigit():
        #     question_info.append({
        #       "question": value,
        #       "question_type": "subjective",
        #       "gpt_prompt_override": "",
        #       "subjective_answer": "",
        #       'question_for': manager_name
        #     })

        for i in range(1,2*num_questions):
              question = {
                      "question_type": "subjective",
                      "gpt_prompt_override": "",
                      "subjective_answer": ""
                  }
              if i % 2 == 0:
                  question['question'] = f"Respond as {manager_name}"
                  question['question_for'] = manager_name
              else:
                  question['question'] = "Please respond in order to continue"
                  question['question_for'] = 'user'
              if i == (2*num_questions-1):
                  question['question'] = "Conclude the discussion as a participant."



              question_info.append(question)

        test_main_context = description + data['Person 0']

        orchestrated_conversation_details = {
              "test_main_context": test_main_context,
              "test_user_persona": data['Candidate Type'].capitalize(),
              "objective": description,
              "initial_messages": [data['Person 0']]

          }


        infomation = {
          'title': title,
          'description': clean_text(description),
          'question_info': question_info,
          "candidate_type": data['Candidate Type'].capitalize(),
          'area_domain': "General",
          'certificate_title': data['Certificate Title'],
          'email_list': data['Email Address List'],
          'responder': data['Responder'],
          'orchestrated_conversation_details': orchestrated_conversation_details,
          'skill_domain': intake.get('Skill Domain', "")
        }


        if data.get('start with user') != "None":
          infomation['start_with_user'] = data['start with user']

        if data.get('skill_list'):
          infomation['skills_list'] = data['skill_list']

        infomation['feedback_script'] = feedback_video_script_template(
                                                        intake.get('Responder'),
                                                        intake.get('Asker'),
                                                        intake.get('Department'),
                                                        intake.get('Industry'),
                                                        intake.get('Objective'),
                                                        intake.get('Domain Skill'),
                                                        '',
                                                        True
                                                        )


        if video_script:
          infomation['video_script'] = video_script
        elif generate_video_script:
          infomation['video_script'] = generate_video_script_via(description)


        from pprint import pprint

        pprint(f'scenario info============================: {infomation}')


        return infomation, True


      text = text.replace('KLS', 'Skills')

      title_pattern = re.compile(r'Title\s*:\s*(.*?)\n', re.DOTALL)
      description_pattern = re.compile(r'Description\s*:\s*(.*?)\nQuestions\s*:', re.DOTALL)



      question_pattern = re.compile(r'Questions\s*:\s*(.+)')

      skill_pattern = re.compile(r'Skills:\s*(.+)')

      rating_pattern = re.compile(r'Rating\s*:\s*(\d+)')

      if not description_pattern.findall(text):
        description_pattern = re.compile(r'Description\s*:\s*(.*?)\n', re.DOTALL)

      if not question_pattern.findall(text):

          question_pattern = re.compile(r'Question\s*:\s*(.+)')




      # Extracting information using regular expressions

      title_match = title_pattern.search(text)

      description_match = description_pattern.search(text)

      questions_match = question_pattern.search(text)

      rating_match = rating_pattern.search(text)
      skill_match = skill_pattern.search(text)



      # If title_pattern doesn't match, try to find the title as the lines before the description

      if not title_match:

          pattern = re.compile(r'^(?:Title\s*:\s*)?(?:"(.*?)"|([^"\n]*))\n*Description\s*:')

          title_match = pattern.search(text)

          if not title_match:

              raise ValueError("Invalid format. Unable to extract the title.")





      if not (title_match and description_match and question_pattern.findall(text)):

          raise ValueError("Invalid format. Unable to extract necessary information.")



      print('skill_match', skill_match)



      title = title_match.group(1).strip()

      description = clean_text(description_match.group(1).strip())

      questions = clean_text(questions_match.group(1).strip())

      rating = int(rating_match.group(1)) if rating_match else 0

      skill_list = clean_text(skill_match.group(1).strip()) if skill_match else None

      question_info = []



      test_main_context = description + questions

      orchestrated_conversation_details = {

              "test_main_context": test_main_context,

              "test_user_persona": candidate_type.capitalize(),

              "objective": description,

              "initial_messages": [questions]

          }



      skills_list_candidate = set()

      for item in get_skills(candidate_type.capitalize()):

              skills_list_candidate.add(item.capitalize())



      evaluation_skill_list = [skill.strip() for skill in sorted(skills_list_candidate)]



      if len(evaluation_skill_list) < 6:

          raise ValueError(f"Skills must have at least 4. Got:  {len(skills_list_candidate)}, {skills_list_candidate}")



      if len(evaluation_skill_list) > 8:


          evaluation_skill_list = evaluation_skill_list[:8]



      evaluation_skill_list = ','.join(evaluation_skill_list)



      manager_name = questions.split(':')[0].strip()

      for i in range(1,2*num_questions):

          question = {

                  "question_type": "subjective",

                  "gpt_prompt_override": "",

                  "subjective_answer": ""

              }



          if i % 2 == 0:

              question['question'] = f"Respond as {manager_name}"

              question['question_for'] = manager_name

          else:

              question['question'] = "Please respond in order to continue"

              question['question_for'] = 'user'



          if i == (2*num_questions-1):

              question['question'] = "Conclude the discussion as a participant."

          elif i == (2*num_questions-2):
                  question['question'] = f'Respond as {manager_name} with this exact statement, "Thats great! Can you please summarize and highlight any potential next step here. "'


          question_info.append(question)



      infomation = {

          'title': title,

          'description': description,

          'question_info': question_info,

          'skill_to_evaluate': evaluation_skill_list,

          'rating': rating,

          'orchestrated_conversation_details': orchestrated_conversation_details,
          'candidate_type': candidate_type,
          'responder': manager_name,
          'certificate_title': title,
          'email_list': 'mail@coachbots.com',
          'area_domain': 'General',
          'skill_domain': intake.get('Skill Domain', "")
      }

      if skill_list:
        infomation['skills_list'] = skill_list
      elif targeted_skills:
          infomation['skills_list'] = targeted_skills

      if video_script:
          infomation['video_script'] = video_script
      elif generate_video_script:
        infomation['video_script'] = generate_video_script_via(description)

      infomation['feedback_script'] = feedback_video_script_template(
                                                        intake.get('Responder'),
                                                        intake.get('Asker'),
                                                        intake.get('Department'),
                                                        intake.get('Industry'),
                                                        intake.get('Objective'),
                                                        intake.get('Domain Skill'),
                                                        '',
                                                        True
                                                        )

      from pprint import pprint

      pprint(f'scenario info============================: {infomation}')
      return infomation, True

    except Exception as e:
      question_info = []
      for i in range(1,2*num_questions):
              question = {
                      "question_type": "subjective",
                      "gpt_prompt_override": "",
                      "subjective_answer": ""
                  }
              if i % 2 == 0:
                  question['question'] = f"Error"
                  question['question_for'] = 'bot'
              else:
                  question['question'] = "Please respond in order to continue"
                  question['question_for'] = 'user'
              if i == (2*num_questions-1):
                  question['question'] = "Conclude the discussion as a participant."



              question_info.append(question)


      orchestrated_conversation_details = {
              "test_main_context": 'test_main_context',
              "test_user_persona": '',
              "objective": 'description',
              "initial_messages": ["person 0"]

          }

      return {'error': str(e), 'question_info': question_info, 'orchestrated_conversation_details': orchestrated_conversation_details}, False



def extract_game_type(text,scenario_type,candidate_type="Manager",num_questions=3,intake=None,video_script=None):
  data = fetch_json_from_string(text)
  if not data:
    raise ValueError("Invalid format. Unable to extract necessary information.")
  print(data['is_single_select'].strip().lower())
  information = {
      "title": data['title'],
      "description": data['description'],
      "game_questions": data['questions'],
      "is_single_select": data['is_single_select'].strip().lower() == 'true',
      "email_list": "mail@coachbots.com"
  }

  from pprint import pprint

  pprint(f'scenario info============================: {information}')

  return information, True


def scenario_create(llm_type,
                    scenario_type,
                    question_count,
                    iterations,
                    skill_count,
                    question_type,
                    personality_model,
                    startwithuser_type,
                    candidate_type,
                    objective,
                    industry,
                    department,
                    responder,
                    asker,
                    situation,
                    targeted_skills,
                    custom_information,
                    skill_domain,
                    generate_video_script,
                    video_script = None,
                    file_name=None):
  is_pitch = False
  if scenario_type == 'pitch':
      question_count = 1
      scenario_type = 'normal_static'
      is_pitch = True

  if question_count <=3 and skill_count <2:
    skill_count = 2


  personality_model = None if personality_model == 'none' else personality_model
  scenario_types = [scenario_type]



  intake = {
      'Skill Domain': skill_domain,
      'Targeted Skills': targeted_skills,
      'Objective': objective,
      'Industry': industry,
      'Department': department,
      'Asker': asker,
      'Responder': responder,
      'Situation': situation
  }


  information = f"""
  Situation: {situation}\n
  Objective: {objective}\n
  Industry: {industry}\n
  Department: {department}\n
  Skill Domain: {skill_domain}\n
  Responder: {responder}\n
  Asker: {asker}\n
  Targeted Skills: {targeted_skills}\n
  """
  if scenario_type == 'dynamic_start_with_user':
    information = f"{information}\nStart with user: {startwithuser_type}\n CandidateType: {candidate_type}"

  if len(custom_information.strip()) > 0:
    information = custom_information

  print("="*100)
  print("Information:  \n",information)

  errors = []
  all_failed = False
  for iteration in range(iterations):
    if 'game' in scenario_type:
      prompt = get_game_prompt(
          industry=industry,
          informatioquestion_countn=information,
          num_of_questions=1 if scenario_type == 'dynamic_game' else question_count,
          question_type=question_type,
          candidate_type=candidate_type
      )
    else:
      prompt = get_scenario_prompt(scenario_type, information, skill_count, question_count,create_skill=len(targeted_skills)>0)
    print(f"iteration: {iteration + 1}, prompt_type: {scenario_type} Starts")
    print("="*100)
    print(f"prompt: {prompt}")
    print("="*100)
    for index in range(3):
      if llm_type == 'gemini':
        raw_scenario = gemini_completion(prompt)
      else:
        raw_scenario = anthropic_completion(prompt)

      print(f"raw_scenario: {raw_scenario}")

      try:
        scenario_information_data = {}
        if 'game' in scenario_type:
          scenario_information_data, success = extract_game_type(text=raw_scenario,scenario_type=scenario_type,num_questions=question_count,intake=intake,generate_video_script=generate_video_script,video_script=video_script)
          if not success:
            raise ValueError(f"Got error: {scenario_information_data.get('error')}")
          dynamic_create_csv(
              scenario_data=scenario_information_data,
              question_count=question_count,
              iteration=iteration,
              scenario_type=scenario_type,
              is_game=True,
              information=intake,
              file_name=file_name
          )
          dynamic_create_csv(
              scenario_data=scenario_information_data,
              question_count=question_count,
              iteration=iteration,
              scenario_type=scenario_type,
              is_game=True,
              information=intake
          )
        elif "static" in scenario_type:
          scenario_information_data, success = extract_information_static(raw_scenario,
                                     iteration+1,
                                     question_count,
                                     scenario_type,
                                     candidate_type,
                                     is_pitch=is_pitch,
                                     personality_model=personality_model,
                                     intake=intake,generate_video_script=generate_video_script,
                                     video_script=video_script
                                      )
          if not success:
            raise ValueError(f"Got error: {scenario_information_data.get('error')}")

          create_csv(scenario_information_data,question_count,iteration,scenario_type,information=intake)
          create_csv(scenario_information_data,question_count,iteration,scenario_type,information=intake,file_name=file_name)

        else:
          scenario_information_data,success = extract_information_dynamic_scenario(text=raw_scenario,
                                               scenario_type=scenario_type,
                                               num_questions=question_count,
                                               targeted_skills=targeted_skills if len(targeted_skills)>0 else None,
                                               intake=intake,
                                               generate_video_script=generate_video_script,
                                               video_script=video_script)


          if not success:
            raise ValueError(f"Got error: {scenario_information_data.get('error')}")

          dynamic_create_csv(scenario_information_data,question_count,iteration,scenario_type,information=intake,file_name=file_name)
          dynamic_create_csv(scenario_information_data,question_count,iteration,scenario_type,information=intake)

        print(f"iteration: {iteration+1} Ends")
        print("="*100)
        break
      except Exception as e:
            # raise e
            print('*'*100)
            print(f'[Got Error Skipping this test for {index+1}]: {e}')
            if index+1 == 3:
              errors.append(f"Failed to create iter {iteration+1} for retry {index+1}, error: {e}  info: {information}")
              all_failed = True
              if "static" in scenario_type:
                create_csv(scenario_information_data,question_count,iteration,scenario_type,information=intake,file_name=file_name)
                create_csv(scenario_information_data,question_count,iteration,scenario_type,information=intake,file_name=None)

              else:
                dynamic_create_csv(scenario_information_data,question_count,iteration,scenario_type,file_name=file_name,information=intake)
                dynamic_create_csv(scenario_information_data,question_count,iteration,scenario_type,file_name=None,information=intake)




            initialize_csv('Failed Scenarios.csv', ['information', 'raw Scenario', 'error'])
            append_to_csv('Failed Scenarios.csv', [information, raw_scenario, f"Failed to create iter {iteration+1} for retry {index+1}, error: {e}"])

  return True if len(errors) == 0 else False, errors

def get_candidate_type(role_key):
    mapping = {
        "team-manager": "Employee",
        "sales-customer": "Sales Manager",
        "customer-sales": "Customer Service",
        "manager-team": "Manager"
    }
    return mapping.get(role_key, "Unknown")

# def initialize_csv(file_name, fields= ["Test Code","TEST TITLE","Client Name", "Test Type","Report Link", "Success" ,"Error"]):
#     # from google.colab import drive
#     # import os

#     # drive.mount('/content/drive')

#     # # Create a folder in Drive
#     # folder_path = '/content/drive/MyDrive/Collab Downloads'
#     # os.makedirs(folder_path, exist_ok=True)

#     # file_path = os.path.join(folder_path, file_name)

#     # Check if file already exists
#     file_exists = os.path.isfile(file_name)

#     # Open the file in append mode and write header if not exists
#     with open(file_name, mode='a', newline='') as file:
#         writer = csv.writer(file)
#         if not file_exists:
#             writer.writerow(fields)



# Function to append a row to the CSV file
# def append_to_csv(file_name, row):
#     folder_path = '/content/drive/MyDrive/Collab Downloads'

#     file_path = os.path.join(folder_path, file_name)
#     with open(file_name, mode='a', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(row)



def create_upload_test_scenario(llm_type,
                    scenario_type,
                    question_count,
                    iterations,
                    skill_count,
                    question_type,
                    personality_model,
                    startwithuser_type,
                    candidate_type,
                    objective,
                    industry,
                    department,
                    responder,
                    asker,
                    situation,
                    targeted_skills,
                    custom_information,
                    skill_domain,
                    generate_video_script,
                    file_name,
                    video_script = None,
                                ):

  # scenairo creation
  test_type = 'dynamic'
  if 'static' in scenario_type:
    file_name = f'bulk_static_{question_count}_que-{file_name}.csv'
    test_type = 'static'

  elif 'game' in scenario_type:
    file_name = f'bulk_game_{question_count}_que-{file_name}.csv'
  else:
    file_name = f'bulk_dynamic_{question_count}_que-{file_name}.csv'


  success, errors = scenario_create(
                    llm_type=llm_type,
                    scenario_type=scenario_type,
                    question_count=question_count,
                    iterations=iterations,
                    skill_count=skill_count,
                    question_type=question_type,
                    personality_model=personality_model,
                    startwithuser_type=startwithuser_type,
                    candidate_type=candidate_type,
                    objective=objective,
                    industry=industry,
                    department=department,
                    asker=asker,
                    responder=responder,
                    situation=situation,
                    targeted_skills=targeted_skills,
                    custom_information=custom_information,
                    skill_domain=skill_domain,
                    generate_video_script=generate_video_script,
                    video_script=video_script,
                    file_name=file_name
                )

  if len(errors)>0:
    return False, errors



  with open(file_name, 'rb') as f:
    try:
      create_and_attempt_test(f,email, password, domain,test_type)
    except Exception as e:
      return False, [f'❌ Failed to Upload and test, Reason: {e}']

  try:
      os.remove(file_name)
  except OSError as e:
      # Optional: log or handle if deletion fails
      print(f"⚠️ Failed to delete file {file_name}: {e}")

  return True, []






#Execution

In [ ]:
# @title Bulk Test Creator
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
import io
import time

# -----------------------
# Styling Helpers
# -----------------------
def make_label(text, bold=True, size='16px'):
    return widgets.HTML(f"<span style='font-size:{size}; font-weight:{'bold' if bold else 'normal'}'>{text}</span>")

# -----------------------
# Widgets
# -----------------------

llm_dropdown2 = widgets.Dropdown(
    options=["anthropic", "gemini"],
    value="gemini",
    description="LLM:",
    style={'description_width': 'initial'}
)
upload_label = make_label("📁 Upload Your CSV File")

upload_widget = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Choose File',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='auto')
)

submit_button = widgets.Button(
    description='🚀 Bulk Create',
    button_style='success',
    tooltip='Click to process CSV',
    icon='check',
    layout=widgets.Layout(width='150px')
)

loader = widgets.Label(
    value='',
    layout=widgets.Layout(visibility='hidden', padding='5px 0px')
)

message_label = widgets.HTML(
    value='',
    layout=widgets.Layout(padding='5px 0px')
)

# Output console (scrollable box)
output_console_2 = widgets.Output()

scrollable_output_box2 = widgets.Box([output_console_2])
scrollable_output_box2.layout = widgets.Layout(
    border='1px solid #ccc',
    padding='10px',
    height='300px',
    overflow_y='scroll',
    overflow_x='auto',
    margin='10px 0px 0px 0px'
)

# CSV storage
uploaded_df = {}

# -----------------------
# Handlers
# -----------------------
def handle_upload(change):
    output_console_2.clear_output()
    loader.value = ''
    loader.layout.visibility = 'hidden'
    uploaded_df.clear()

    for filename, file_info in upload_widget.value.items():
        global_file_name = f'Bulk Report Testing - {datetime.datetime.today().strftime("%Y-%m-%d")}-{filename} .csv'

        content = io.StringIO(file_info['content'].decode('utf-8'))
        df = pd.read_csv(content)
        df = df.where(pd.notna(df), "")
        uploaded_df['df'] = df
        uploaded_df['filename'] = filename

upload_widget.observe(handle_upload, names='value')
required_columns = [
    "Test Type", "Objective", "Industry", "Department", "Asker", "Responder",
    "Situation", "Targeted Skills", "Iterations", "Pesonality Model",
    "start with user", "Is single select", "Skill Domain", "Generate Video Script", "Video Script"
]

def on_submit_click(b):
    output_console_2.clear_output()
    message_label.value = ''
    if uploaded_df.get('df') is not None:
        loader.value = '⏳ Processing...'
        loader.layout.visibility = 'visible'

        df = uploaded_df['df']
        filename = uploaded_df['filename'].replace('.csv','')

        with output_console_2:
            # --- Check Required Columns ---
            missing = [col for col in required_columns if col not in df.columns]
            if missing:
                loader.layout.visibility = 'hidden'
                print("❌ Missing required columns:\n")
                for m in missing:
                    print(f"• {m}")
                print("\n⚠️ Please re-upload a valid CSV with all required fields.")
                return

            print("✅ CSV contains all required fields. Processing rows...\n")
            llm_type = llm_dropdown2.value
            for index, row in df.iterrows():
                index = index + 1
                # Simulate processing for each row
                row = row.to_dict()
                print(f"Processing row {index}: {row}")
                objective = row['Objective']
                industry = row['Industry']
                department = row['Department']
                asker = row['Asker']
                responder = row['Responder']
                situation = row['Situation']
                targeted_skills = row['Targeted Skills'] if row['Targeted Skills'] else ""
                skill_domain = row['Skill Domain']
                generate_video_script = str(row.get('Generate Video Script')).lower() == 'true' if row.get('Generate Video Script') else False

                video_script = row['Video Script']
                custom_information = ""
                scenario_type = row['Test Type']
                question_count = 6
                try:
                  iterations = int(row['Iterations']) if row['Iterations'] else 1
                except:
                  iterations = 1
                question_type = ""
                skill_count = 2

                # if targeted_skills and len(targeted_skills.split(',')) < 8:
                #   message_label.value += f"<span style='color:red;'>Row {index}: Skills must have at least 8. Got:  {len(targeted_skills.split(','))}, {targeted_skills}</span><br>"
                #   continue
                personality_model = row['Pesonality Model'] if row['Pesonality Model'] else None

                if scenario_type == "dynamic_start_with_user":
                  if not row['start with user']:
                    message_label.value += f"<span style='color:red;'>Row {index}: For {scenario_type}, field 'start with user' required.</span><br>"
                    print(f"Row {index}: For {scenario_type}, field 'start with user required.")
                    continue
                if "game" in scenario_type:
                  if not row['Is single select']:
                    message_label.value += f"<span style='color:red;'>Row {index}: For {scenario_type}, field 'Is single select' required.</span><br>"
                    print(f"Row {index}: For {scenario_type}, field 'Is single select' required.")
                    continue
                  question_type = 'single' if row['Is single select'] else "multiple"

                startwithuser_type = row['start with user'] if scenario_type == "dynamic_start_with_user" else ""
                candidate_type = get_candidate_type(row['start with user']) if scenario_type == "dynamic_start_with_user" else "Manager"

                success, errors = create_upload_test_scenario(
                    llm_type=llm_type,
                    scenario_type=scenario_type,
                    question_count=question_count,
                    iterations=iterations,
                    skill_count=skill_count,
                    question_type=question_type,
                    personality_model=personality_model,
                    startwithuser_type=startwithuser_type,
                    candidate_type=candidate_type,
                    objective=objective,
                    industry=industry,
                    department=department,
                    asker=asker,
                    responder=responder,
                    situation=situation,
                    targeted_skills=targeted_skills,
                    custom_information=custom_information,
                    skill_domain=skill_domain,
                    generate_video_script=generate_video_script,
                    video_script=video_script,
                    file_name =filename
                )
                print(errors)
                if len(errors)>0:
                  error_msg = "<br>".join(errors)
                  message_label.value += f"<span style='color:red;'>⚠️ Got error in row {index}: {error_msg}.</span><br>"
                else:
                  message_label.value += f"<span style='color:green;'>✅ Row {index} processed successfully.</span><br>"



            print("✅ Bulk Test Created Successfully!")
            loader.layout.visibility = 'hidden'
            message_label.value += f"<span style='color:green;'>✅ Bulk Test Created Successfully!</span><br>"
            download_file(global_file_name)

    else:
        with output_console_2:
            message_label.value = "<span style='color:red;'>⚠️ Please upload a CSV file first.</span>"
            print("⚠️ Please upload a CSV file before submitting.")

submit_button.on_click(on_submit_click)

# -----------------------
# Form Layout
# -----------------------
form = widgets.VBox([
    make_label("🧾 Bulk Test Creation, Upload, Test", size='20px'),
    widgets.HBox([upload_label]),
    widgets.HBox([llm_dropdown2]),
    widgets.HBox([upload_widget]),
    widgets.HBox([submit_button]),
    message_label,
    loader,
], layout=widgets.Layout(
    border='1px solid #ccc',
    padding='20px',
    border_radius='10px',
    width='60%',
    align_items='flex-start'
))

# Wrapper to center form
form_wrapper = widgets.HBox([form], layout=widgets.Layout(justify_content='center'))

# -----------------------
# Display Everything
# -----------------------
display(form_wrapper)
display(scrollable_output_box2)


Box(children=(Output(),), layout=Layout(border='1px solid #ccc', height='300px', margin='10px 0px 0px 0px', ov…